In [170]:
import sys
from pathlib import Path

# Get the current notebook's directory
parent = Path().resolve()  # Current working directory (notebook's directory)
root = parent.parent.parent       # Go one level up

print("Current Directory:", parent)
print("Parent Directory:", root)

# Optionally append to sys.path
sys.path.append(str(root))


Current Directory: D:\mukul\Desktop\Unimelb\Masters\Capstone\Learning-Dynamic-Systems\indirect_identification\notebooks
Parent Directory: D:\mukul\Desktop\Unimelb\Masters\Capstone\Learning-Dynamic-Systems


# param to theta

In [2]:
import numpy as np
n_states = 4
n_inputs = 2
n_output = 2
noise_order = 1
n_params = n_states + n_states*n_inputs + n_output*(noise_order+1)
print("total_params:", n_params)
C = np.array([[0,0,0,1],[0,1,0,0]])
def _construct_ss_from_params(params,C):
    """
    Returns state space matrices A_obs,B_obs,C_obs,D_obs and the A,B polynomials
    """
    # A: n_state x n_state matrix
    A =  params[:n_states]
    A_obs = np.hstack([np.vstack([np.zeros(n_states-1), np.eye(n_states-1)]), -np.flipud(A.reshape(A.size,-1))])
    # B: n_state x n_input matrix
    B = params[n_states:n_states+n_states*n_inputs].reshape(n_inputs,n_states)
    B_obs = np.flipud(B.T)
    # C: n_output x n_state matrix
    C_obs = C
    # D: n_output x n_input matrix: zero matrix for now
    D_obs = np.zeros((n_output,n_inputs))

    A = np.hstack([1, A])
    B = np.hstack([np.zeros((n_inputs,1)), B])

    return A_obs, B_obs, C_obs, D_obs, A,B
    


params = np.array([1,2,3,4,11,12,13,14,21,22,23,24,31,32,33,34])
A_obs, B_obs, C_obs, D_obs, A,B = _construct_ss_from_params(params,C)
print("A_obs: ", A_obs)
print("B_obs:",  B_obs)
print("C_obs: ", C_obs)
print("A",A)
print("B", B)

total_params: 16
A_obs:  [[ 0.  0.  0. -4.]
 [ 1.  0.  0. -3.]
 [ 0.  1.  0. -2.]
 [ 0.  0.  1. -1.]]
B_obs: [[14 24]
 [13 23]
 [12 22]
 [11 21]]
C_obs:  [[0 0 0 1]
 [0 1 0 0]]
A [1 1 2 3 4]
B [[ 0. 11. 12. 13. 14.]
 [ 0. 21. 22. 23. 24.]]


In [3]:
C = np.empty((n_output, noise_order+1))
if n_output == 1:
    # if only one output, H is a scalar transfer function
    # H = np.zeros((n_states, n_states), dtype=object)
    j = n_states + n_states*n_inputs
    num = params[j:j+(noise_order+1)]
    den = A
    C[0]=num
    print(num, den)
else:
    H = np.zeros((n_output, n_output), dtype=object)
    for i in range(n_output):
        j = n_states + n_states*n_inputs + i*(noise_order+1)
        # numerator and denominator of the transfer function
        num = params[j:j+noise_order+1]
        den = A
        C[i]=num
        print("num:",num)
        print("den",den)
print(C)

num: [31 32]
den [1 1 2 3 4]
num: [33 34]
den [1 1 2 3 4]
[[31. 32.]
 [33. 34.]]


# SISO

In [4]:
import numpy as np
def create_phi_optimized(Y: np.ndarray, U: np.ndarray, n_a: int, n_b: int, c: float) -> np.ndarray:
    """
    Create the phi matrix optimized for the given inputs.
    
    Parameters:
    Y (array): The output sequence.
    U (array): The input sequence.
    n_a (int): The number of lags for the output sequence.
    n_b (int): The number of lags for the input sequence.
    
    Returns:
    array: The phi matrix.
    """
    t = Y.shape[0]
    
    # Initialize phi with zeros
    phi = np.zeros(shape=[t, n_a + n_b], dtype=Y.dtype)
    
    # Handle Y lags
    for i in range(1, n_a + 1):
        phi[i:, i-1] = -Y[:-i]/c
    
    # Handle U lags
    for i in range(1, n_b + 1):
        phi[i:, n_a+i-1] = U[:-i]/c
    
    return phi

Y = np.random.rand(10)
U = np.random.rand(10)
print(Y)

phi = create_phi_optimized(Y,U,2,3,2)
print(phi)
print(phi.shape)


from numba import njit
@njit()
def create_phi_optimized_n(Y: np.ndarray, U: np.ndarray, n_a: int, n_b: int, c: float) -> np.ndarray:
    """
    Create the phi matrix optimized for the given inputs.
    
    Parameters:
    Y (array): The output sequence.
    U (array): The input sequence.
    n_a (int): The number of lags for the output sequence.
    n_b (int): The number of lags for the input sequence.
    
    Returns:
    array: The phi matrix.
    """
    t = Y.shape[0]
    
    # Initialize phi with zeros
    phi = np.zeros((t, n_a + n_b), dtype=Y.dtype)
    
    # Handle Y lags
    for i in range(1, n_a + 1):
        phi[i:, i-1] = -Y[:-i]/c
    
    # Handle U lags
    for i in range(1, n_b + 1):
        phi[i:, n_a+i-1] = U[:-i]/c
    
    return phi


@njit(cache=True)
def create_phi_optimized_n2(Y: np.ndarray, U: np.ndarray, n_a: int, n_b: int, c: float) -> np.ndarray:
    """
    Create the phi matrix optimized for the given inputs using Numba-friendly loops.
    
    Parameters:
    Y (array): The output sequence.
    U (array): The input sequence.
    n_a (int): Number of Y lags.
    n_b (int): Number of U lags.
    c (float): Normalization constant.
    
    Returns:
    array: The phi matrix.
    """
    t = Y.shape[0]
    phi = np.zeros((t, n_a + n_b), dtype=Y.dtype)
    # Output lags (Y)
    for lag in range(1, n_a + 1):
        for i in range(lag, t):
            phi[i, lag - 1] = -Y[i - lag] / c
    # Input lags (U)
    for lag in range(1, n_b + 1):
        for i in range(lag, t):
            phi[i, n_a + lag - 1] = U[i - lag] / c
    return phi
phi = create_phi_optimized_n(Y,U,2,3,2)
print(phi)
print(phi.shape)

ph2 = create_phi_optimized_n2(Y,U,2,3,2)
print(ph2)
print(np.all(phi==ph2))

from cupyx.profiler import benchmark

print(benchmark(create_phi_optimized,(Y,U,2,3,2), n_repeat=1000))
print(benchmark(create_phi_optimized_n,(Y,U,2,3,2), n_repeat=10000))
print(benchmark(create_phi_optimized_n2,(Y,U,2,3,2), n_repeat=10000))

[0.9848867  0.44510671 0.80962354 0.04233853 0.24096281 0.30661465
 0.06513046 0.80681941 0.91641656 0.05988858]
[[ 0.          0.          0.          0.          0.        ]
 [-0.49244335  0.          0.33698732  0.          0.        ]
 [-0.22255335 -0.49244335  0.43731552  0.33698732  0.        ]
 [-0.40481177 -0.22255335  0.15442171  0.43731552  0.33698732]
 [-0.02116927 -0.40481177  0.33416984  0.15442171  0.43731552]
 [-0.12048141 -0.02116927  0.17877367  0.33416984  0.15442171]
 [-0.15330732 -0.12048141  0.30737928  0.17877367  0.33416984]
 [-0.03256523 -0.15330732  0.30158894  0.30737928  0.17877367]
 [-0.4034097  -0.03256523  0.22453096  0.30158894  0.30737928]
 [-0.45820828 -0.4034097   0.36183483  0.22453096  0.30158894]]
(10, 5)
[[ 0.          0.          0.          0.          0.        ]
 [-0.49244335  0.          0.33698732  0.          0.        ]
 [-0.22255335 -0.49244335  0.43731552  0.33698732  0.        ]
 [-0.40481177 -0.22255335  0.15442171  0.43731552  0.336987

In [5]:
from scipy.signal import lfilter
from indirect_identification.tf_methods.fast_tfs_methods_fast_math import _convolve
@njit()
def create_phi_optimized_noise(Y: np.ndarray, U: np.ndarray, N: np.ndarray, n_a: int, n_b: int, n_c: int) -> np.ndarray:
    """
    Create the phi matrix optimized for the given inputs using Numba-friendly loops.
    
    Parameters:
    Y (array): The output sequence.
    U (array): The input sequence.
    n_a (int): Number of Y lags.
    n_b (int): Number of U lags.
    c (float): Normalization constant.
    
    Returns:
    array: The phi matrix.
    """
    t = Y.shape[0]
    n_a = n_a -1
    n_b = n_b -1
    n_c = n_c 
    print(n_a,n_b,n_c)
    phi = np.zeros((t, n_a + n_b +n_c ), dtype=Y.dtype)
    print(phi.shape)
    # Output lags (Y)
    for lag in range(1, n_a+1):
        for i in range(lag, t):
            phi[i, lag - 1] = Y[i - lag]
    # Input lags (U)
    for lag in range(1, n_b+1):
        for i in range(lag, t):
            phi[i, n_a + lag - 1] = -U[i - lag] 
    for lag in range(n_c+1):
        for i in range(lag, t):
            phi[i, n_a+n_b+lag]= -N[i-lag]
    return phi.T
# AR, X, and MA coefficients
A = np.array([1, 0.3, -0.2])    # a1, a2
B = np.array([0, 0.5, 0.1])     # b1, b2
C = np.array([2])          # c1

A_Y = lfilter(A,[1], Y)
B_U = lfilter(B,[1], U)
eps_t = A_Y - B_U
phi = create_phi_optimized_noise(Y,U,eps_t, len(A),len(B),len(C))
print(phi)

for i in range(len(A)+len(B)-3):
    phi[i] = lfilter([1],C, phi[i])

C2 = _convolve(C,C)
for i in range(len(C)):
    phi[-i-1] = lfilter([1], C2, phi[-i-1])

print(phi)

2 2 1
(10, 5)
[[ 0.          0.9848867  -0.9848867  -0.4035854  -0.24146522  0.04568057
   0.27331452 -0.12482815  0.23421172 -0.40197082]
 [ 0.          0.          0.9848867   0.44510671  0.80962354  0.04233853
   0.24096281  0.30661465  0.06513046  0.80681941]
 [ 0.         -0.67397464 -0.87463105 -0.30884343 -0.66833968 -0.35754734
  -0.61475856 -0.60317788 -0.44906192 -0.72366967]
 [ 0.          0.         -0.67397464 -0.87463105 -0.30884343 -0.66833968
  -0.35754734 -0.61475856 -0.60317788 -0.44906192]
 [-0.9848867  -0.4035854  -0.24146522  0.04568057  0.27331452 -0.12482815
   0.23421172 -0.40197082 -0.86058754  0.23329136]]
[[ 0.          0.49244335 -0.49244335 -0.2017927  -0.12073261  0.02284028
   0.13665726 -0.06241407  0.11710586 -0.20098541]
 [ 0.          0.          0.49244335  0.22255335  0.40481177  0.02116927
   0.12048141  0.15330732  0.03256523  0.4034097 ]
 [ 0.         -0.33698732 -0.43731552 -0.15442171 -0.33416984 -0.17877367
  -0.30737928 -0.30158894 -0.2245309

In [6]:
import numpy as np
from numba import njit
from cupyx.profiler import benchmark
@njit(cache=True, fastmath=True)
def lfilter_numba(b, a, x):
    N = len(a)
    M = len(b)
    n = len(x)

    if a[0] != 1.0:
        b = b / a[0]
        a = a / a[0]

    y = np.zeros(n)

    for i in range(n):
        for j in range(M):
            if i - j >= 0:
                y[i] += b[j] * x[i - j]
        for j in range(1, N):
            if i - j >= 0:
                y[i] -= a[j] * y[i - j]

    return y

from scipy.signal import lfilter

# Example filter coefficients
b = np.array([0.1, 0.2, 0.3])
a = np.array([1.0, -0.3, 0.2])

# Input signal
x = np.random.randn(1000)

# Compare with scipy
y_scipy = lfilter(b, a, x)
y_numba = lfilter_numba(b, a, x)

# Check difference
print("Max error:", np.max(np.abs(y_scipy - y_numba)))
print("Median error:",  np.median(np.abs(y_scipy - y_numba)))
print(benchmark(lfilter,(b, a, x), n_repeat=10000))
print(benchmark(lfilter_numba,(b, a, x), n_repeat=10000))

Max error: 3.3306690738754696e-16
Median error: 2.0816681711721685e-17
lfilter             :    CPU:    30.179 us   +/- 12.910 (min:    21.700 / max:   399.200) us     GPU-0:    54.676 us   +/- 77.492 (min:     2.208 / max:  2274.528) us
lfilter_numba       :    CPU:    14.422 us   +/-  4.315 (min:    10.600 / max:   106.000) us     GPU-0:    27.931 us   +/- 40.417 (min:     2.208 / max:  1448.544) us


In [7]:
from scipy.signal import lfilter
from indirect_identification.tf_methods.fast_tfs_methods_fast_math import _convolve
from numba import prange
@njit(cache=True, fastmath=True)
def create_phi_optimized_filter(Y: np.ndarray, U: np.ndarray, A: np.ndarray, B: np.ndarray, C:np.ndarray) -> np.ndarray:
    """
    Create the phi matrix optimized for the given inputs using Numba-friendly loops.
    
    Parameters:
    Y (array): The output sequence.
    U (array): The input sequence.
    n_a (int): Number of Y lags.
    n_b (int): Number of U lags.
    c (float): Normalization constant.
    
    Returns:
    array: The phi matrix.
    """
    t = Y.shape[0]
    n_a = len(A) -1
    n_b = len(B) -1
    n_c = len(C) 
    num = np.array([1.0])
    phi = np.zeros((t, n_a + n_b +n_c ), dtype=Y.dtype)
    filtered_Y = lfilter_numba(num, C, Y)
    filtered_U = lfilter_numba(num, C, U)
    A_Y = lfilter_numba(A,C, filtered_Y)
    B_U = lfilter_numba(B,C, filtered_U)
    eps_t = A_Y - B_U
    # Output lags (Y)
    for lag in range(1, n_a+1):
        for i in range(lag, t):
            phi[i, lag - 1] = filtered_Y[i - lag]
    # Input lags (U)
    for lag in range(1, n_b+1):
        for i in range(lag, t):
            phi[i, n_a + lag - 1] = -filtered_U[i - lag] 
    for lag in range(n_c):
        for i in range(lag, t):
            phi[i, n_a+n_b+lag]= -eps_t[i-lag]
    return phi

@njit(cache=True,fastmath=True)
def create_phi_optimized_filter2(Y, U, A, B, C):
    t = Y.shape[0]
    n_a = len(A) - 1
    n_b = len(B) - 1
    n_c = len(C)

    idx_b = n_a
    idx_c = n_a + n_b

    phi = np.zeros((t, idx_c + n_c), dtype=np.float64)

    ones = np.empty(1, dtype=np.float64)
    ones[0] = 1.0

    filtered_Y = lfilter_numba(ones, C, Y)
    filtered_U = lfilter_numba(ones, C, U)
    A_Y = lfilter_numba(A, C, filtered_Y)
    B_U = lfilter_numba(B, C, filtered_U)
    eps_t = A_Y - B_U

    for i in range(t):
        if i > 0:
            for lag in range(1, n_a + 1):
                    phi[i, lag - 1] = filtered_Y[i - lag]
            for lag in range(1, n_b + 1):
                    phi[i, idx_b + lag - 1] = -filtered_U[i - lag]
        for lag in range(n_c):
                phi[i, idx_c + lag] = -eps_t[i - lag]

    return phi


# AR, X, and MA coefficients
A = np.array([1.0, 0.3, -0.2])    # a1, a2
B = np.array([0, 0.5, 0.1])     # b1, b2
C = np.array([2.0])          # c1


print(Y)

phi = create_phi_optimized_filter(Y,U, A, B, C)
print(phi)
phi = create_phi_optimized_filter2(Y,U, A, B, C)
print(phi)


print(benchmark(create_phi_optimized_filter,(Y,U, A, B, C), n_repeat=100000))
print(benchmark(create_phi_optimized_filter2,(Y,U, A, B, C), n_repeat=100000))


[0.9848867  0.44510671 0.80962354 0.04233853 0.24096281 0.30661465
 0.06513046 0.80681941 0.91641656 0.05988858]
[[ 0.          0.          0.          0.         -0.24622167]
 [ 0.49244335  0.         -0.33698732  0.         -0.10089635]
 [ 0.22255335  0.49244335 -0.43731552 -0.33698732 -0.06036631]
 [ 0.40481177  0.22255335 -0.15442171 -0.43731552  0.01142014]
 [ 0.02116927  0.40481177 -0.33416984 -0.15442171  0.06832863]
 [ 0.12048141  0.02116927 -0.17877367 -0.33416984 -0.03120704]
 [ 0.15330732  0.12048141 -0.30737928 -0.17877367  0.05855293]
 [ 0.03256523  0.15330732 -0.30158894 -0.30737928 -0.10049271]
 [ 0.4034097   0.03256523 -0.22453096 -0.30158894 -0.21514689]
 [ 0.45820828  0.4034097  -0.36183483 -0.22453096  0.05832284]]
[[ 0.          0.          0.          0.         -0.24622167]
 [ 0.49244335  0.02994429 -0.33698732 -0.430415   -0.10089635]
 [ 0.22255335  0.49244335 -0.43731552 -0.33698732 -0.06036631]
 [ 0.40481177  0.22255335 -0.15442171 -0.43731552  0.01142014]
 [ 0

In [8]:

print(benchmark(create_phi_optimized_filter,(Y,U, A, B, C), n_repeat=1000000))
print(benchmark(create_phi_optimized_filter2,(Y,U, A, B, C), n_repeat=1000000))


KeyboardInterrupt: 

## all perturbed phi: m x t x (n_a+n_b+n_c)

In [9]:
from numba import njit
import numpy as np

@njit(cache=True, fastmath=True)
def create_phi_optimized_filter_multi(Y, U, A, B, C):
    m, t = Y.shape  # m outputs, t time steps
    n_a = len(A) - 1
    n_b = len(B) - 1
    n_c = len(C)

    idx_b = n_a
    idx_c = n_a + n_b
    total_phi_len = n_a + n_b + n_c

    phi = np.zeros((m, t, total_phi_len), dtype=np.float64)

    ones = np.empty(1, dtype=np.float64)
    ones[0] = 1.0

    # Input filtered once (shared across outputs)
    filtered_U = lfilter_numba(ones, C, U)
    B_U = lfilter_numba(B, C, filtered_U)

    for j in range(m):  # loop over each output dimension
        y = Y[j]

        filtered_Y = lfilter_numba(ones, C, y)
        A_Y = lfilter_numba(A, C, filtered_Y)
        eps_t = A_Y - B_U

        for i in range(t):
            if i > 0:
                for lag in range(1, n_a + 1):
                    if i - lag >= 0:
                        phi[j, i, lag - 1] = filtered_Y[i - lag]
                for lag in range(1, n_b + 1):
                    if i - lag >= 0:
                        phi[j, i, idx_b + lag - 1] = -filtered_U[i - lag]
            for lag in range(n_c):
                if i - lag >= 0:
                    phi[j, i, idx_c + lag] = -eps_t[i - lag]

    return phi

m = 2
t=3
alpha = np.sign(np.random.randn(m,t))
alpha[0, :] = 1
Y = np.multiply(alpha, np.random.rand(t))
U =  np.random.rand(t)
# AR, X, and MA coefficients
A = np.array([1.0, 0.3, -0.2])    # a1, a2
B = np.array([0, 0.5, 0.1])     # b1, b2
C = np.array([1.0])          # c1
print("Y: ", Y)
phi = create_phi_optimized_filter_multi(Y,U,A,B,C)
print(phi)
print(phi.shape)

Y:  [[0.66536029 0.96895839 0.77537826]
 [0.66536029 0.96895839 0.77537826]]
[[[ 0.          0.          0.          0.         -0.66536029]
  [ 0.66536029  0.         -0.32683074  0.         -1.00515111]
  [ 0.96895839  0.66536029 -0.09200026 -0.32683074 -0.85431052]]

 [[ 0.          0.          0.          0.         -0.66536029]
  [ 0.66536029  0.         -0.32683074  0.         -1.00515111]
  [ 0.96895839  0.66536029 -0.09200026 -0.32683074 -0.85431052]]]
(2, 3, 5)


In [10]:
@njit(cache=True, fastmath=True)
def create_phi_optimized_multi(Y: np.ndarray, U: np.ndarray, len_a: int, len_b: int, c: float) -> np.ndarray:
    """
    Create the phi matrix for perturbed output (Y: m x t) and single input (U: t).
    
    Parameters:
    Y (array): Output matrix of shape (m, t).
    U (array): Input vector of shape (t,).
    len_a (int): len of A polynomial
    len_b (int): len of B polynomial
    c (float): Normalization constant.
    
    Returns:
    array: Phi matrix of shape (m, t, n_a + n_b).
    """
    n_a=len_a-1
    n_b=len_b-1
    m, t = Y.shape
    phi = np.zeros((m, t, n_a + n_b), dtype=Y.dtype)

    for j in range(m):  # for each output dimension
        for lag in range(1, n_a + 1):
            for i in range(lag, t):
                phi[j, i, lag - 1] = -Y[j, i - lag] / c

    for lag in range(1, n_b + 1):
        for i in range(lag, t):
            for j in range(m):  # input is shared across outputs
                phi[j, i, n_a + lag - 1] = U[i - lag] / c

    return phi

phi = create_phi_optimized_multi(Y,U,len(A),len(B),1)
print(phi)
print(benchmark(create_phi_optimized_multi,(Y,U,len(A),len(B),1)))

[[[ 0.          0.          0.          0.        ]
  [-0.66536029  0.          0.32683074  0.        ]
  [-0.96895839 -0.66536029  0.09200026  0.32683074]]

 [[ 0.          0.          0.          0.        ]
  [-0.66536029  0.          0.32683074  0.        ]
  [-0.96895839 -0.66536029  0.09200026  0.32683074]]]
create_phi_optimized_multi:    CPU:    11.406 us   +/- 36.833 (min:     2.200 / max:  3023.000) us     GPU-0:    50.704 us   +/- 178.842 (min:     2.176 / max: 10534.944) us


# MIMO

## 2 input 1 output

In [11]:
import numpy as np
from numba import njit
from cupyx.profiler import benchmark
A = np.array([1.0, 0.3, -0.2, 0.1, 0.2])    # a1, a2
B = np.array([0.0, 0.5, 0.1, 0.2, 0.1])     # b1, b2
C = np.array([[1.0, 0.5],[1.0, 0.8]])          # c1


@njit(fastmath=True)
def _get_p_F(A, B):
    P = np.empty(5, dtype=np.float64)

    A1, A2, A3, A4 = A[1], A[2], A[3], A[4]
    B1, B2, B3, B4 = B[1], B[2], B[3], B[4]
    P[0] = 0.0
    P[1] = B3
    P[2] = B4 + A1 * B3 - A3 * B1
    P[3] = A1 * B4 + A2 * B3 - A3 * B2 - A4 * B1
    P[4] = A2 * B4 - A4 * B2
    return P



print(benchmark(_get_p_F, (A,B), n_repeat=10000))

_get_p_F            :    CPU:     5.209 us   +/-  4.996 (min:     2.600 / max:   247.900) us     GPU-0:    23.020 us   +/- 56.283 (min:     2.176 / max:  2278.816) us


In [12]:
import numpy as np
from numba import njit, float64
from cupyx.profiler import benchmark
@njit(cache=True,fastmath=True)
def create_phi_optimized_2states_1input(Y, U, A, B, C):
    m, n_o, t = Y.shape
    J = np.zeros((m, t, n_o, 4))

    a1, a2 = A[1], A[2]
    b1, b2 = B[1], B[2]

    for k in range(m):
        Y1 = Y[k, 0]
        Y2 = Y[k, 1]

        for i in range(t):
            Jk0 = J[k, i, 0]
            Jk1 = J[k, i, 1]

            if i >= 2:
                Jk0[0] = Y1[i - 1]
                Jk0[1] = Y1[i - 2]
                Jk0[2] = -U[i - 1]
                Jk0[3] = -U[i - 2]

                Jk1[0] = Y2[i - 1] - b2 * U[i - 2]
                Jk1[1] = Y2[i - 2] + b1 * U[i - 2]
                Jk1[2] = a2 * U[i - 2]
                Jk1[3] = -(U[i - 1] + a1 * U[i - 2])
            elif i == 1:
                Jk0[0] = Y1[i - 1]
                Jk0[2] = -U[i - 1]

                Jk1[0] = Y2[i - 1]
                Jk1[3] = -U[i - 1]
            elif i == 0:
                pass  # J already zero

    return J

@njit(cache=True, fastmath=True)
def create_phi_optimized_2states_1input2(Y, U, A, B, C):
    m, _, t = Y.shape
    J = np.zeros((m, t, 2, 4), dtype=np.float64)

    a1: float64 = A[1]
    a2: float64 = A[2]
    b1: float64 = B[1]
    b2: float64 = B[2]

    for k in range(m):
        Y1 = Y[k, 0]
        Y2 = Y[k, 1]

        # i = 1
        if t > 1:
            i = 1
            im1 = i - 1

            y1_im1 = Y1[im1]
            y2_im1 = Y2[im1]
            u_im1 = U[im1]

            J[k, i, 0, 0] = y1_im1
            J[k, i, 0, 2] = -u_im1
            J[k, i, 1, 0] = y2_im1
            J[k, i, 1, 3] = -u_im1

        # i = 2 to t
        for i in range(2, t):
            im1 = i - 1
            im2 = i - 2
            Jk0 = J[k, i, 0]
            Jk1 = J[k, i, 1]

            y1_im1 = Y1[im1]
            y1_im2 = Y1[im2]
            y2_im1 = Y2[im1]
            y2_im2 = Y2[im2]
            u_im1 = U[im1]
            u_im2 = U[im2]

            # Row 0
            Jk0[0] = y1_im1
            Jk0[1] = y1_im2
            Jk0[2] = -u_im1
            Jk0[3] = -u_im2

            # Row 1
            Jk1[0] = y2_im1 - b2 * u_im2
            Jk1[1] = y2_im2 + b1 * u_im2
            Jk1[2]= a2 * u_im2
            Jk1[3]= -(u_im1 + a1 * u_im2)

    return J
@njit(cache=True, fastmath=True)
def create_phi_optimized_2states_1input3(Y, U, A, B, C):
    m, _, t = Y.shape
    
    # Normalize U to always be (m, t)
    if U.ndim == 1:
        U_eff = np.broadcast_to(U, (m, U.shape[0]))
    elif U.shape[0] == 1:
        U_eff = np.broadcast_to(U[0], (m, U.shape[1]))
    else:
        U_eff = U

    J = np.zeros((m, t, 2, 4))

    a1 = A[1]
    a2 = A[2]
    b1 = B[1]
    b2 = B[2]

    for k in range(m):
        Y1 = Y[k, 0]
        Y2 = Y[k, 1]
        Uk = U_eff[k]

        if t > 1:
            Jk0 = J[k, 1, 0]
            Jk1 = J[k, 1, 1]

            Jk0[0] = Y1[0]
            Jk0[2] = -Uk[0]
            Jk1[0] = Y2[0]
            Jk1[3] = -Uk[0]

        for i in range(2, t):
            im1 = i - 1
            im2 = i - 2

            Jk0 = J[k, i, 0]
            Jk1 = J[k, i, 1]

            # Row 0
            Jk0[0] = Y1[im1]
            Jk0[1] = Y1[im2]
            Jk0[2] = -Uk[im1]
            Jk0[3] = -Uk[im2]

            # Row 1
            Jk1[0] = Y2[im1] - b2 * Uk[im2]
            Jk1[1] = Y2[im2] + b1 * Uk[im2]
            Jk1[2] = a2 * Uk[im2]
            Jk1[3] = -(Uk[im1] + a1 * Uk[im2])

    return J

@njit(cache=True, fastmath=True)
def create_phi_optimized_2states_1input4(Y, U, A, B, C):
    m, _, t = Y.shape
    
    # Broadcasting U to m rows
    if U.ndim == 1:
        U_eff = np.broadcast_to(U, (m, U.shape[0]))
    elif U.shape[0] == 1:
        U_eff = np.broadcast_to(U[0], (m, U.shape[1]))
    else:
        U_eff = U
    

    # Preallocate J once with the correct size
    J = np.zeros((m, t, 2, 4), dtype=np.float64)
    
    # Cache commonly used values from A and B
    a1, a2 = A[1], A[2]
    b1, b2 = B[1], B[2]

    for k in range(m):
        Y1 = Y[k, 0]
        Y2 = Y[k, 1]
        Uk = U_eff[k]  # This is already broadcasted to (t, )

        # Handle i = 1 separately (no im2 index)
        if t > 1:
            Jk0 = J[k, 1, 0]
            Jk1 = J[k, 1, 1]

            Jk0[0] = Y1[0]
            Jk0[2] = -Uk[0]
            Jk1[0] = Y2[0]
            Jk1[3] = -Uk[0]

        # Loop for i = 2 to t
        for i in range(2, t):
            im1, im2 = i - 1, i - 2
            Jk0 = J[k, i, 0]
            Jk1 = J[k, i, 1]

            # Reusing values instead of multiple calls to Y and U
            y1_im1, y1_im2 = Y1[im1], Y1[im2]
            y2_im1, y2_im2 = Y2[im1], Y2[im2]
            u_im1, u_im2 = Uk[im1], Uk[im2]

            # Assign values to Jk0 and Jk1 directly without intermediate variables
            Jk0[0] = y1_im1
            Jk0[1] = y1_im2
            Jk0[2] = -u_im1
            Jk0[3] = -u_im2

            Jk1[0] = y2_im1 - b2 * u_im2
            Jk1[1] = y2_im2 + b1 * u_im2
            Jk1[2] = a2 * u_im2
            Jk1[3] = -(u_im1 + a1 * u_im2)

    return J



m = 2
t=5
n_o=2
alpha = np.sign(np.random.randn(m,n_o,t))
alpha[0, :] = 1
Y = np.multiply(alpha, np.random.rand(n_o,t))
print(Y.shape)
U =  np.random.rand(t)
print(U.shape)
# AR, X, and MA coefficients
A = np.array([1.0, 0.3, -0.2])    # a1, a2
B = np.array([0, 0.5, 0.1])     # b1, b2
C = np.array([1.0])          # c1

phi = create_phi_optimized_2states_1input(Y,U,A,B,C)
print(phi.shape)

functions = [
    create_phi_optimized_2states_1input,
    create_phi_optimized_2states_1input2,
    create_phi_optimized_2states_1input3,
    create_phi_optimized_2states_1input4,
]

for f in functions:
    p = f(Y,U,A,B,C)
    print(np.allclose(phi, p))
    print(benchmark(f, (Y, U, A, B, C), n_repeat=100000))


(2, 2, 5)
(5,)
(2, 5, 2, 4)
True
create_phi_optimized_2states_1input:    CPU:     5.753 us   +/-  6.584 (min:     3.200 / max:  1316.100) us     GPU-0:    23.342 us   +/- 64.092 (min:     2.144 / max:  3934.112) us
True
create_phi_optimized_2states_1input2:    CPU:     5.844 us   +/-  5.243 (min:     3.000 / max:   467.900) us     GPU-0:    22.710 us   +/- 60.267 (min:     2.144 / max:  2076.672) us
True
create_phi_optimized_2states_1input3:    CPU:     5.551 us   +/-  4.551 (min:     3.400 / max:   472.700) us     GPU-0:    20.295 us   +/- 52.238 (min:     2.176 / max:  1883.136) us
True
create_phi_optimized_2states_1input4:    CPU:     5.283 us   +/-  3.162 (min:     3.100 / max:   172.200) us     GPU-0:    19.569 us   +/- 50.028 (min:     2.208 / max:  1813.440) us


In [13]:

from typing import Tuple, List, Union
import numpy as np
from scipy.signal import lfilter
from dB.sim_db import Database, SPSType
from indirect_identification.d_tfs import d_tfs
from types import SimpleNamespace
from indirect_identification.sps_utils import get_phi_method

np.random.seed(42)

class SPS_indirect_model:
    """
    Indirect Sign Perturbed Sum (SPS) model class.
    """
    def __init__(self, m: int, q: int, n_inputs: int, n_outputs: int, n_noise: int, N: int = 50):
        """
        Initialize the SPS model.
        assuming siso or full state obsrevation 
        Parameters:
        m (int): The number of perturbations.
        q (int): The number of retained perturbations.
        N (int): The length of the perturbation sequence.
        """
        self.N = N
        self.m = m
        self.q = q
        self.n_inputs = n_inputs
        self.n_outputs = n_outputs
        if self.n_outputs == 1:
            self.alpha = np.random.randn(m,N)
            self.alpha = np.sign(self.alpha)
            self.alpha[0, :] = 1
        else:
            self.alpha = np.sign(np.random.randn(m,n_o,t))
            self.alpha[0, :] = 1
        self.pi_order = np.random.permutation(np.arange(m))

        self.create_phi_optimized = get_phi_method(n_inputs, n_outputs, n_noise)
    

    def transform_to_open_loop(self, 
                               G: Union['d_tfs', Tuple[Union[List[float], np.ndarray], Union[List[float], np.ndarray]]], 
                               H: Union['d_tfs', Tuple[Union[List[float], np.ndarray], Union[List[float], np.ndarray]]], 
                               F: Union['d_tfs', Tuple[Union[List[float], np.ndarray], Union[List[float], np.ndarray]]], 
                               L: Union['d_tfs', Tuple[Union[List[float], np.ndarray], Union[List[float], np.ndarray]]]) -> Tuple['d_tfs', 'd_tfs']:
        """
        Transform the closed-loop system to an open-loop system.
        
        Parameters:
        G (tuple): The transfer function G.
        H (tuple): The transfer function H.
        F (tuple): The transfer function F.
        L (tuple): The transfer function L.
        
        Returns:
        tuple: The open-loop transfer functions G_0 and H_0.
        """
        if not isinstance(G, d_tfs):
            G = d_tfs(G)
        if not isinstance(H, d_tfs):
            H = d_tfs(H)
        if not isinstance(F, d_tfs):
            F = d_tfs(F)
        if not isinstance(L, d_tfs):
            L = d_tfs(L)

        GF_plus_I = (G * F) + 1
        i_GF_plus_I = 1/GF_plus_I
        
        if not all(tf.is_stable() for tf in [L, G, H, 1/H, i_GF_plus_I] if isinstance(tf, d_tfs)):
            raise ValueError(f"Error transforming to open loop: stability conditions not satisfied.")
        
        G_0 = i_GF_plus_I * G * L
        H_0 = i_GF_plus_I * H
        return G_0, H_0
    
    def open_loop_sps(self, G_0, H_0, A: np.ndarray, B: np.ndarray, C: np.ndarray, Y_t: np.ndarray, U_t: np.ndarray) -> Tuple[bool, np.ndarray]:
        """
        Perform open-loop SPS.
        
        Parameters:
        G_0 (d_tfs): The open-loop transfer function G_0.
        H_0 (d_tfs): The open-loop transfer function H_0.
        Y_t (array): The output sequence.
        U_t (array): The input sequence.
        n_a (int): The number of lags for the output sequence.
        n_b (int): The number of lags for the input sequence.
        
        Returns:
        tuple: A boolean indicating if the rank is within the threshold and the S values.
        """
        try:
            Y_t = np.asarray(Y_t)
            U_t = np.asarray(U_t)
            YGU = Y_t - G_0*U_t
            N_hat = (1/H_0)*YGU
            
            # Extract relevant segments
            N_hat_par = N_hat[-self.N:]
            U_t_par = U_t[-self.N-1:-1]
            perturbed_N_hat = np.multiply(self.alpha, N_hat_par)
            
            # Compute y_bar vectorized
            y_bar = G_0*U_t_par[None, :] + H_0*(perturbed_N_hat[:, None])
            y_bar = y_bar.transpose(1, 0, 2)[0]
            # Compute phi_tilde
            phi_tilde = self.create_phi_optimized(y_bar, U_t_par, A, B, C)
            # Compute Cholesky decomposition and norm squared
            R = np.matmul(phi_tilde.transpose(0, 2, 1), phi_tilde) / len(Y_t)
            L = np.linalg.cholesky(R)
            Q, R = np.linalg.qr(L)
            R_root_inv = np.linalg.solve(R, Q.transpose(0, 2, 1))
            weighted_sum = np.matmul(phi_tilde.transpose(0, 2, 1), perturbed_N_hat[:, :, None])
            S = np.sum(np.square(np.matmul(R_root_inv, weighted_sum)), axis=(1, 2))
            # Ranking
            combined = np.array(list(zip(self.pi_order, S)))
            order = np.lexsort(combined.T)
            rank_R = np.where(order == 0)[0][0] + 1
            
            return rank_R <= self.m - self.q, S
        except Exception as e:
            raise ValueError(f"Error in open-loop SPS: {e}")
        
    def open_loop_sps_mimo(self, G_0, H_0, A: np.ndarray, B: np.ndarray, C: np.ndarray, Y_t: np.ndarray, U_t: np.ndarray):
        pass


In [39]:
from indirect_identification.d_tfs import apply_tf_matrix, invert_matrix
from indirect_identification.d_tfs import d_tfs


def apply_tf_matrix2(G: np.ndarray, U: np.ndarray) -> np.ndarray:
    """
    Applies a matrix of transfer functions G to an input matrix U.

    This function performs element-wise multiplication between each transfer function 
    in G and the corresponding row in U, then accumulates the results along the rows.

    Parameters:
        G (np.ndarray): A (m x n) matrix where each element is a transfer function 
            (or a scalar/matrix that supports multiplication with U).
        U (np.ndarray): A (n x k) matrix representing n input signals, each of length k.

    Returns:
        np.ndarray: A (m x k) matrix where each row represents the output of 
        applying the corresponding transfer functions from G to the inputs in U.

    Example:
        >>> G = np.array([[tf1, tf2], [tf3, tf4]])  # Some transfer functions
        >>> U = np.array([[u1, u2, u3], [v1, v2, v3]])  # Two input signals
        >>> Y = apply_tf_matrix(G, U)
        >>> print(Y.shape)  # (2, 3), output has same time length as U
    """
    if U.ndim == 2:
        # in this case we are simply applying tf to a 2d input matrix
        m, n = G.shape  # m outputs, n inputs
        k = U.shape[-1]  # Time length of each input sequence
        Y = np.zeros((m, k))  # Initialize output matrix
        for i in range(m):
            for j in range(n):
                Y[i, :] += (G[i, j] * U[j, :]).reshape((k,))  # Apply transfer function multiplication
    elif U.ndim == 3 and G.shape[0]==G.shape[1]:
        # m x n_output x t to noutput x m x t
        # this is H, which is assumed to be all on diagonal 
        U =np.ascontiguousarray(U.transpose(1, 0, 2))
        n,m,t = U.shape
        Y = np.empty_like(U)
        for i in range(n):
            H = G[i,i]
            for j in range(n):
                Y[j,:] = H*U[j,:]
        Y=Y.transpose(1,0,2)
    return Y



n_states = 2
n_inputs = 1
n_output = 2
noise_order = -1
n_params = n_states + n_states*n_inputs + n_output*(noise_order+1)
print("total_params:", n_params)
C_obs = np.array([[0,1],[1,0]])

m=100
q=5
t=100
N=50
alpha = np.sign(np.random.randn(m,n_output,N))
alpha[0, :] = 1
Y = np.random.rand(n_output,t)
U =  np.random.rand(n_inputs, t)
phi_method = get_phi_method(n_inputs=n_inputs, n_outputs=n_output, n_noise=-1)
pi_order = np.random.permutation(np.arange(m))

@njit
def compute_phi_phiT(phi_tilde):
    m, T, r, c = phi_tilde.shape  # (3, 5, 2, 4)
    result = np.zeros((m, r, r))
    for k in range(m):
        for t in range(T):
            phi = phi_tilde[k, t]  # shape (2, 4)
            # phi @ phi.T -> (2, 2)
            for i in range(r):
                for j in range(r):
                    for l in range(c):  # inner dimension
                        result[k, i, j] += phi[i, l] * phi[j, l]
    result/=T
    return result

@njit
def compute_phi_Y(phi, Y):
    m, t, r, c = phi.shape  # (3, 5, 4, 2)
    result = np.zeros((m, r, 1))  # (3, 4, 1)

    for i in range(m):       # systems
        for j in range(t):   # time
            for k in range(r):     # output dim (4)
                acc = 0.0
                for l in range(c): # inner dim (2)
                    acc += phi[i, j, k, l] * Y[i, l, j]  # note: Y is (m, 2, t)
                result[i, k, 0] += acc
    return result


def open_loop_sps_mimo(G_0, H_0, A: np.ndarray, B: np.ndarray, C: np.ndarray, Y_t: np.ndarray, U_t: np.ndarray):
    YGU = Y_t - apply_tf_matrix2(G_0, U_t)
    H_invert = np.zeros_like(H_0)
    for i in range(n_output):
        H_invert[i,i]=d_tfs((A, C[i]))
    N_hat = apply_tf_matrix2(H_invert,YGU)
    print(N_hat.shape)
    N_hat_par = N_hat[:, -N:]
    U_t_par = U_t[:, -N-1:-1]
    perturbed_N_hat = np.multiply(alpha, N_hat_par)
    print("calculating y_bar")
    apply_tf_matrix2(H_0, perturbed_N_hat[:]) 
    y_bar = apply_tf_matrix2(G_0, U_t_par) + apply_tf_matrix2(H_0, perturbed_N_hat[:]) 
    phi_tilde = phi_method(y_bar, U_t_par, A, B, C)
    print(phi_tilde.transpose(0,2, 3, 1).shape, phi_tilde.shape)
    R = compute_phi_phiT(phi_tilde)
    print(R.shape)
    L = np.linalg.cholesky(R)
    Q, R = np.linalg.qr(L)
    R_root_inv = np.linalg.solve(R, Q.transpose(0, 2, 1))
    print( perturbed_N_hat.shape)
    weighted_sum = compute_phi_Y(phi_tilde, perturbed_N_hat)
    S = np.sum(np.square(np.matmul(R_root_inv, weighted_sum)), axis=(1, 2))
    combined = np.array(list(zip(pi_order, S)))
    order = np.lexsort(combined.T)
    rank_R = np.where(order == 0)[0][0] + 1
    return rank_R <= m - q

def _construct_ss_from_params(params,C_obs):
    """
    Returns state space matrices A_obs,B_obs,C_obs,D_obs and the A,B polynomials
    """
    # A: n_state x n_state matrix
    A =  params[:n_states]
    A_obs = np.hstack([np.vstack([np.zeros(n_states-1), np.eye(n_states-1)]), -np.flipud(A.reshape(A.size,-1))])
    # B: n_state x n_input matrix
    B = params[n_states:n_states+n_states*n_inputs].reshape(n_inputs,n_states)
    B_obs = np.flipud(B.T)
    # D: n_output x n_input matrix: zero matrix for now
    D_obs = np.zeros((n_output,n_inputs))

    A = np.hstack([1, A])
    B = np.hstack([np.zeros((n_inputs,1)), B])

    return A_obs, B_obs, C_obs, D_obs, A,B
    


params = np.array([0.1, -1.0, 0.1, 0.3])
A_obs, B_obs, C_obs, D_obs, A,B = _construct_ss_from_params(params,C_obs)
G = d_tfs.ss_to_tf(A_obs, B_obs, C_obs, D_obs, check_assumption=False)
C = np.empty((n_output, 1))
H = np.zeros((n_output, n_output), dtype=object)
for i in range(n_output):
    C[i]=np.array([1.0])
    H[i,i]=d_tfs((np.array([1.0]),A))
print("A_obs: ", A_obs)
print("B_obs:",  B_obs)
print("C_obs: ", C_obs)
print("A",A)
print("B", B)
print(G)
print(H)
print(C)
# print("Y:", Y)
print("U:", U.shape)
print(G[0,0])
print(G[0,0]*U)
open_loop_sps_mimo(G, H, A, B, C, Y, U)

total_params: 4
A_obs:  [[ 0.   1. ]
 [ 1.  -0.1]]
B_obs: [[0.3]
 [0.1]]
C_obs:  [[0 1]
 [1 0]]
A [ 1.   0.1 -1. ]
B [[0.  0.1 0.3]]
[[Transfer Function: num=[0.  0.1 0.3], den=[ 1.   0.1 -1. ]]
 [Transfer Function: num=[0.   0.3  0.13], den=[ 1.   0.1 -1. ]]]
[[Transfer Function: num=[1.], den=[ 1.   0.1 -1. ] 0]
 [0 Transfer Function: num=[1.], den=[ 1.   0.1 -1. ]]]
[[1.]
 [1.]]
U: (1, 100)
Transfer Function: num=[0.  0.1 0.3], den=[ 1.   0.1 -1. ]
[[ 0.          0.01343391  0.10435905  0.21056123  0.14539155  0.3542777
   0.40268452  0.61392484  0.66942531  0.80926302  0.93503029  1.01742238
   1.05506719  1.23903797  1.16364019  1.23649452  1.25812868  1.26486662
   1.29068695  1.36531773  1.39616016  1.62085841  1.59225754  1.68169383
   1.5552074   1.72554906  1.51482752  1.908265    1.637825    1.85671359
   1.61322698  1.78193427  1.53469703  1.97956831  1.59571145  2.1551037
   1.69132093  2.03853262  1.58302818  2.12936256  1.61770101  2.11930636
   1.58140442  2.08816678  1

False

In [38]:
@njit( fastmath=True)
def lfilter_1d(b, a, x):
    N = len(a)
    M = len(b)
    n = len(x)
    y = np.zeros(n)

    if a[0] != 1.0:
        b = b / a[0]
        a = a / a[0]

    for i in range(n):
        for j in range(M):
            if i - j >= 0:
                y[i] += b[j] * x[i - j]
        for j in range(1, N):
            if i - j >= 0:
                y[i] -= a[j] * y[i - j]

    return y

@njit( fastmath=True)
def lfilter_nd(b, a, x_reshaped, n_signals, time_len):
    y_reshaped = np.empty_like(x_reshaped)
    for i in range(n_signals):
        y_reshaped[i] = lfilter_1d(b, a, x_reshaped[i])
    return y_reshaped

@njit( fastmath=True)
def lfilter_numba2(b, a, x):
    orig_shape = x.shape
    time_len = x.shape[-1]

    # Collapse all leading dims to 1D signals
    x_reshaped = x.reshape(-1, time_len)
    y_reshaped = lfilter_nd(b, a, x_reshaped, x_reshaped.shape[0], time_len)

    # Restore original shape
    return y_reshaped.reshape(orig_shape)
@njit( fastmath=True)
def lfilter_numba3(b, a, x):
    x = np.asarray(x)
    shape = x.shape
    ndim = x.ndim
    time_len = shape[-1]
    y = np.empty_like(x)

    # Iterate over all indices except the last one (time axis)
    it = np.ndindex(*shape[:-1])
    for idx in it:
        y[idx] = lfilter_1d(b, a, x[idx])

    return y
rand_input = np.random.rand(100, 50)
print(lfilter_numba2(G[0,0].num, G[0,0].den, rand_input).shape)
print(np.allclose(lfilter_numba2(G[0,0].num, G[0,0].den, rand_input), lfilter(G[0,0].num, G[0,0].den, rand_input)))
print(benchmark(lfilter_numba2, (G[0,0].num, G[0,0].den, rand_input), n_repeat=10000))
print(benchmark(lfilter, (G[0,0].num, G[0,0].den, rand_input), n_repeat=10000))
print(benchmark(lfilter_numba3, (G[0,0].num, G[0,0].den, rand_input), n_repeat=10000))

(100, 50)
True
lfilter_numba2      :    CPU:    87.372 us   +/- 25.062 (min:    64.200 / max:   621.200) us     GPU-0:   119.931 us   +/- 149.662 (min:     2.208 / max:  2459.712) us
lfilter             :    CPU:   103.194 us   +/- 32.293 (min:    69.300 / max:   430.300) us     GPU-0:   135.600 us   +/- 125.176 (min:     2.272 / max:  1852.384) us
lfilter_numba3      :    CPU:    87.015 us   +/- 22.453 (min:    51.800 / max:   293.300) us     GPU-0:   117.494 us   +/- 133.007 (min:     2.176 / max:  1959.200) us


In [125]:
import numpy as np
from numba import njit, prange
from cupyx.profiler import benchmark

def compute_cov_matrices_np(epsilon):
    d, N = epsilon.shape

    # Covariance matrix Lambda = 1/(N-1) * sum (eps_centered @ eps_centered.T)
    Lambda = np.cov(epsilon)

    # Second moment matrix Lambda_n = 1/N * sum (epsilon @ epsilon.T)
    Lambda_n = epsilon @ epsilon.T / N

    # Inverse of covariance
    Lambda_inv = np.linalg.inv(Lambda)

    result = Lambda_inv @ Lambda_n @ Lambda_inv

    return Lambda, Lambda_n, result

@njit(fastmath=True)
def compute_cov_matrices(epsilon):
    d, N = epsilon.shape

    # Covariance matrix Lambda = 1/(N-1) * sum (eps_centered @ eps_centered.T)
    Lambda = np.cov(epsilon)

    # Second moment matrix Lambda_n = 1/N * sum (epsilon @ epsilon.T)
    Lambda_n = epsilon @ epsilon.T / N

    # Inverse of covariance
    Lambda_inv = np.linalg.inv(Lambda)

    result = Lambda_inv @ Lambda_n @ Lambda_inv

    return Lambda, Lambda_n, result

@njit(fastmath=True)
def compute_cov_matrices_loop(epsilon):
    d, N = epsilon.shape

    # Step 1: Compute Lambda (covariance matrix)
    # Lambda = 1 / (N-1) * epsilon_centered @ epsilon_centered.T

    # First, compute mean of epsilon over N
    mean_epsilon = np.zeros(d)
    for i in range(d):
        for n in range(N):
            mean_epsilon[i] += epsilon[i, n]
        mean_epsilon[i] /= N

    # Compute centered epsilon
    epsilon_centered = np.zeros((d, N))
    for i in range(d):
        for n in range(N):
            epsilon_centered[i, n] = epsilon[i, n] - mean_epsilon[i]

    # Now compute epsilon_centered @ epsilon_centered.T
    Lambda = np.zeros((d, d))
    for i in range(d):
        for j in range(d):
            sum_ = 0.0
            for n in range(N):
                sum_ += epsilon_centered[i, n] * epsilon_centered[j, n]
            Lambda[i, j] = sum_ / (N - 1)


    # Step 2: Compute Lambda_n = 1/N * epsilon @ epsilon.T
    Lambda_n = np.zeros((d, d))
    for i in range(d):
        for j in range(d):
            sum_ = 0.0
            for n in range(N):
                sum_ += epsilon[i, n] * epsilon[j, n]
            Lambda_n[i, j] = sum_ / N

    # Step 3: Compute Lambda_inv
    Lambda_inv = np.linalg.inv(Lambda)

    # Step 4: Compute result = Lambda_inv @ Lambda_n @ Lambda_inv
    # First compute temp = Lambda_inv @ Lambda_n
    temp = np.zeros((d, d))
    for i in range(d):
        for j in range(d):
            for k in range(d):
                temp[i, j] += Lambda_inv[i, k] * Lambda_n[k, j]

    # Now compute result = temp @ Lambda_inv
    result = np.zeros((d, d))
    for i in range(d):
        for j in range(d):
            for k in range(d):
                result[i, j] += temp[i, k] * Lambda_inv[k, j]

    return Lambda, Lambda_n, result


@njit(fastmath=True)
def compute_cov_matrices_loop1(epsilon):
    d, N = epsilon.shape

    # Step 1: Compute mean of epsilon over N
    mean_epsilon = np.zeros(d)
    for i in range(d):
        for n in range(N):
            mean_epsilon[i] += epsilon[i, n]
        mean_epsilon[i] /= N

    # Step 2: Centered epsilon
    epsilon_centered = np.zeros((d, N))
    for i in range(d):
        for n in range(N):
            epsilon_centered[i, n] = epsilon[i, n] - mean_epsilon[i]

    # Step 3: Covariance matrix Lambda
    Lambda = np.zeros((d, d))
    for i in range(d):
        for j in range(d):
            for n in range(N):
                Lambda[i, j] += epsilon_centered[i, n] * epsilon_centered[j, n]
            Lambda[i, j] /= (N - 1)

    # Step 4: Second moment matrix Lambda_n
    Lambda_n = np.zeros((d, d))
    for i in range(d):
        for j in range(d):
            for n in range(N):
                Lambda_n[i, j] += epsilon[i, n] * epsilon[j, n]
            Lambda_n[i, j] /= N

    # Step 5: Inverse of Lambda
    Lambda_inv = np.linalg.inv(Lambda)

    # Step 6: Fused matrix product: result = Lambda_inv @ Lambda_n @ Lambda_inv
    result = np.zeros((d, d))
    for i in range(d):
        for j in range(d):
            for k in range(d):
                for l in range(d):
                    result[i, j] += Lambda_inv[i, k] * Lambda_n[k, l] * Lambda_inv[l, j]

    return Lambda, Lambda_n, result



@njit(fastmath=True)
def compute_cov_matrices_loop2(epsilon):
    d, N = epsilon.shape


    Lambda = np.cov(epsilon)

    # Step 2: Compute Lambda_n = 1/N * epsilon @ epsilon.T
    Lambda_n = np.zeros((d, d))
    for i in range(d):
        for j in range(d):
            sum_ = 0.0
            for n in range(N):
                sum_ += epsilon[i, n] * epsilon[j, n]
            Lambda_n[i, j] = sum_ / N

    # Step 3: Compute Lambda_inv
    Lambda_inv = np.linalg.inv(Lambda)

    # Step 4: Compute result = Lambda_inv @ Lambda_n @ Lambda_inv
    # First compute temp = Lambda_inv @ Lambda_n
    temp = np.zeros((d, d))
    for i in range(d):
        for j in range(d):
            for k in range(d):
                temp[i, j] += Lambda_inv[i, k] * Lambda_n[k, j]

    # Now compute result = temp @ Lambda_inv
    result = np.zeros((d, d))
    for i in range(d):
        for j in range(d):
            for k in range(d):
                result[i, j] += temp[i, k] * Lambda_inv[k, j]

    return Lambda, Lambda_n, result

# Example: 2 variables, 50 time points
eps = np.random.randn(2, 500)

functions = [
    compute_cov_matrices,
    compute_cov_matrices_loop,
    compute_cov_matrices_loop1,
    compute_cov_matrices_loop2,
]

Lambda1, Lambda_n1, expression1 = compute_cov_matrices(eps)
print(benchmark(compute_cov_matrices, (eps,), n_repeat=10000))

for func in functions:
    Lambda, Lambda_n, expression = func(eps)
    print(np.allclose(Lambda, Lambda1), np.allclose(Lambda_n, Lambda_n1), np.allclose(expression, expression1))
    print(benchmark(func, (eps,), n_repeat=10000))

compute_cov_matrices:    CPU:    21.135 us   +/-  4.221 (min:    15.700 / max:    90.400) us     GPU-0:    43.044 us   +/- 73.966 (min:     2.240 / max:  1730.944) us
True True True
compute_cov_matrices:    CPU:    21.424 us   +/-  3.922 (min:    18.000 / max:   116.100) us     GPU-0:    44.825 us   +/- 84.464 (min:     2.208 / max:  2076.640) us
True True True
compute_cov_matrices_loop:    CPU:    11.647 us   +/-  5.157 (min:     7.900 / max:    99.200) us     GPU-0:    33.741 us   +/- 77.981 (min:     2.240 / max:  1822.592) us
True True True
compute_cov_matrices_loop1:    CPU:    11.670 us   +/-  7.900 (min:     6.100 / max:   142.700) us     GPU-0:    32.286 us   +/- 62.549 (min:     2.208 / max:  1669.632) us
True True True
compute_cov_matrices_loop2:    CPU:    18.082 us   +/-  4.656 (min:    13.200 / max:   105.200) us     GPU-0:    39.374 us   +/- 79.661 (min:     2.240 / max:  1868.864) us
compute_cov_matrices_loop:    CPU:    12.033 us   +/-  6.387 (min:     8.200 / max:   38

In [127]:
print(benchmark(compute_cov_matrices_loop, (eps,), n_repeat=1000000))
print(benchmark(compute_cov_matrices_loop1, (eps,), n_repeat=1000000))

compute_cov_matrices_loop:    CPU:    12.095 us   +/-  5.887 (min:     6.300 / max:   924.800) us     GPU-0:    34.543 us   +/- 49.193 (min:     2.176 / max:  8810.400) us
compute_cov_matrices_loop1:    CPU:    12.281 us   +/-  6.405 (min:     7.200 / max:  1507.100) us     GPU-0:    35.381 us   +/- 47.606 (min:     0.448 / max:  6588.224) us


In [46]:
import numpy as np
from numba import njit, prange
from cupyx.profiler import benchmark
# Original function (with for loops)
@njit(cache=True, fastmath=True)
def compute_phi_phiT_original(phi_tilde):
    m, T, r, c = phi_tilde.shape  # (3, 5, 4, 2)
    result = np.zeros((m, r, r))
    
    for k in range(m):
        for t in range(T):
            phi = phi_tilde[k, t]  # shape (r, c)
            # phi @ phi.T -> (r, r)
            for i in range(r):
                for j in range(r):
                    for l in range(c):  # inner dimension
                        result[k, i, j] += phi[i, l] * phi[j, l]
    result /= T
    return result


# Optimized function (using @)
@njit(cache=True, fastmath=True)
def compute_phi_phiT_optimized(phi_tilde):
    m, T, r, c = phi_tilde.shape  # (3, 5, 4, 2)
    result = np.zeros((m, r, r))
    
    for k in range(m):
        for t in range(T):
            phi = phi_tilde[k, t]  # shape (r, c)
            result[k] += np.dot(phi, phi.T)  # (r, c) @ (c, r) -> (r, r)

    result /= T
    return result

# Original function with parallelization using prange
@njit(cache=True, fastmath=True, parallel=True)
def compute_phi_phiT_original_parallel(phi_tilde):
    m, T, r, c = phi_tilde.shape  # (3, 5, 4, 2)
    result = np.zeros((m, r, r))
    
    # Parallelizing the outer loop with prange
    for k in prange(m):  # Using prange for parallelization
        for t in range(T):
            phi = phi_tilde[k, t]  # shape (r, c)
            # phi @ phi.T -> (r, r)
            for i in range(r):
                for j in range(r):
                    for l in range(c):  # inner dimension
                        result[k, i, j] += phi[i, l] * phi[j, l]
    result /= T
    return result


# Optimized function with @ and parallelization using prange
@njit(cache=True, fastmath=True, parallel=True)
def compute_phi_phiT_optimized_parallel(phi_tilde):
    m, T, r, c = phi_tilde.shape  # (3, 5, 4, 2)
    result = np.zeros((m, r, r))
    
    # Parallelizing the outer loop with prange
    for k in prange(m):  # Using prange for parallelization
        for t in prange(T):
            phi = phi_tilde[k, t]  # shape (r, c)
            phi_t = phi.T
            result[k] += np.dot(phi, phi_t)  # (r, c) @ (c, r) -> (r, r)

    result /= T
    return result

# Generate random test data
phi_tilde = np.random.randn(100, 500, 4, 2)

# Compute results from both functions
result_original = compute_phi_phiT_original(phi_tilde)
result_optimized = compute_phi_phiT_optimized(phi_tilde)

# Compare the results
comparison = np.allclose(result_original, result_optimized)

# Output the result of the comparison
print(f"Are the results close? {comparison}")


# Benchmark the optimized function
print(benchmark(compute_phi_phiT_optimized, (phi_tilde,), n_repeat=100))
print(benchmark(compute_phi_phiT_original, (phi_tilde,), n_repeat=100))
print(benchmark(compute_phi_phiT_optimized_parallel, (phi_tilde,), n_repeat=100))
print(benchmark(compute_phi_phiT_original_parallel, (phi_tilde,), n_repeat=100))

Are the results close? True
compute_phi_phiT_optimized:    CPU: 34609.832 us   +/- 9476.963 (min: 25543.000 / max: 80125.800) us     GPU-0: 35260.591 us   +/- 9469.024 (min: 26463.968 / max: 80820.259) us
compute_phi_phiT_original:    CPU:  3205.467 us   +/- 403.888 (min:  2419.200 / max:  4122.200) us     GPU-0:  3860.786 us   +/- 535.423 (min:  2123.552 / max:  5407.168) us
compute_phi_phiT_optimized_parallel:    CPU: 49060.707 us   +/- 2712.959 (min: 45527.800 / max: 58341.200) us     GPU-0: 49932.051 us   +/- 2710.107 (min: 45400.192 / max: 59201.534) us
compute_phi_phiT_original_parallel:    CPU:  2446.931 us   +/- 556.085 (min:  1836.100 / max:  4643.000) us     GPU-0:  3234.975 us   +/- 647.831 (min:  1433.536 / max:  5420.608) us


In [164]:
from numba import njit, prange
import numpy as np

@njit(cache=True, fastmath=True)
def compute_phi_lambda_phiT(phi_tilde, Lambda):
    m, T, r, c = phi_tilde.shape  # (m, T, r, c)
    result = np.zeros((m, r, r))
    for k in range(m):
        for t in range(T):
            phi = phi_tilde[k, t]  # shape (r, c)
            result[k] += phi @ Lambda @ phi.T  # (r, c) @ (c, r) -> (r, r)
    result /= T
    return result
@njit(cache=True, fastmath=True, parallel=True)
def compute_phi_lambda_phiT_manual(phi_tilde, Lambda):
    m, T, r, c = phi_tilde.shape  # (m, T, r, c)
    result = np.zeros((m, r, r))

    for k in prange(m):
        for t in range(T):
            for i in range(r):         # row of phi
                phi_k_t_i = phi_tilde[k, t, i]  # shape (c,)
                for j in range(r):     # row of phi.T (col of phi)
                    val = 0.0
                    phi_k_t_j = phi_tilde[k, t, j]  # shape (c,)
                    for l in range(c):     # col of phi, row of Lambda
                        phi_tilde_k_t_i_l = phi_k_t_i[l]  # shape (c,)
                        for q in range(c): # col of Lambda
                            val += phi_tilde_k_t_i_l * Lambda[l, q] * phi_k_t_j[q]
                    result[k, i, j] += val

    result /= T
    return result

@njit(cache=True, fastmath=True, parallel=True)
def compute_phi_lambda_phiT2(phi_tilde, Lambda):
    m, T, r, c = phi_tilde.shape  # (m, T, r, c)
    result = np.zeros((m, r, r))
    
    for k in prange(m):
        for t in range(T):
            phi = phi_tilde[k, t]  # (r, c)

            for i in range(r):
                for j in range(r):
                    sum_ = 0.0
                    for l in range(c):
                        for p in range(c):
                            sum_ += phi[i, l] * Lambda[l, p] * phi[j, p]
                    result[k, i, j] += sum_

    result /= T
    return result


@njit(cache=True, fastmath=True, parallel=True)
def compute_phi_lambda_phiT_flat(phi_tilde, Lambda):
    m, T, r, c = phi_tilde.shape
    result = np.zeros((m, r, r))

    # Total number of iterations for (k, t)
    total = m * T

    for idx in prange(total):
        k = idx // T
        t = idx % T

        for i in range(r):
            phi_k_t_i = phi_tilde[k, t, i]
            for j in range(r):
                phi_k_t_j = phi_tilde[k, t, j]
                val = 0.0
                for l in range(c):
                    phi_il = phi_k_t_i[l]
                    for q in range(c):
                        val += phi_il * Lambda[l, q] * phi_k_t_j[q]
                result[k, i, j] += val

    result /= T
    return result


@njit(cache=True, fastmath=True, parallel=True)
def compute_phi_lambda_phiT_flat_all(phi_tilde, Lambda):
    m, T, r, c = phi_tilde.shape
    result = np.zeros((m, r, r))

    total = m * T * r

    for index in prange(total):
        k = index // (T * r)
        t = (index % (T * r)) // r
        i = index % r

        phi_k_t_i = phi_tilde[k, t, i]
        for j in range(r):
            phi_k_t_j = phi_tilde[k, t, j]
            val = 0.0
            for l in range(c):
                phi_il = phi_k_t_i[l]
                for q in range(c):
                    val += phi_il * Lambda[l, q] * phi_k_t_j[q]
            result[k, i, j] += val

    result /= T
    return result

def compute_phi_lambda_phiT_numpy(phi_tilde, Lambda):
    # phi_tilde: (m, T, r, c)
    # Lambda: (c, c)
    phi_lambda = phi_tilde @ Lambda               # (m, T, r, c) @ (c, c) -> (m, T, r, c)
    result = np.matmul(phi_lambda, phi_tilde.transpose(0, 1, 3, 2))  # (m, T, r, r)
    result = result.mean(axis=1)                  # average over T
    return result

def compute_phi_lambda_phiT_numpy2(phi_tilde, Lambda):
    m, T, r, c = phi_tilde.shape  # (m, T, r, c)
    result = np.zeros((m, r, r))
    for k in range(m):
        for t in range(T):
            phi = phi_tilde[k, t]  # shape (r, c)
            result[k] += phi @ Lambda @ phi.T  # (r, c) @ (c, r) -> (r, r)
    result /= T
    return result

# Generate random test data
phi_tilde = np.random.randn(100, 500, 4, 2)
# Lambda matrix
eps = np.random.randn(2, 500)
Lambda, Lambda_n= compute_cov_matrices(eps)

functions = [
    compute_phi_lambda_phiT,
    compute_phi_lambda_phiT2,
    compute_phi_lambda_phiT_manual,
    compute_phi_lambda_phiT_flat,
    compute_phi_lambda_phiT_flat_all,
    compute_phi_lambda_phiT_numpy,
    compute_phi_lambda_phiT_numpy2
]

plp1 = compute_phi_lambda_phiT(phi_tilde, Lambda)

for func in functions:
    if func != compute_phi_lambda_phiT:
        plp = func(phi_tilde, Lambda)
        print(f"{func.__name__}: ", np.allclose(plp1, plp))

# Benchmark the optimized function
for func in functions:
    print(benchmark(func, (phi_tilde, Lambda), n_repeat=100))


compute_phi_lambda_phiT2:  True
compute_phi_lambda_phiT_manual:  True
compute_phi_lambda_phiT_flat:  True
compute_phi_lambda_phiT_flat_all:  True
compute_phi_lambda_phiT_numpy:  True
compute_phi_lambda_phiT_numpy2:  True
compute_phi_lambda_phiT:    CPU: 53725.052 us   +/- 8416.164 (min: 41550.100 / max: 75583.300) us     GPU-0: 54407.758 us   +/- 8452.618 (min: 42120.670 / max: 76380.478) us
compute_phi_lambda_phiT2:    CPU:  4125.669 us   +/- 959.098 (min:  2165.900 / max:  7412.400) us     GPU-0:  4872.529 us   +/- 988.264 (min:  2123.072 / max:  8118.336) us
compute_phi_lambda_phiT_manual:    CPU:  5343.521 us   +/- 935.155 (min:  3258.100 / max: 10217.300) us     GPU-0:  6109.956 us   +/- 1055.270 (min:  3651.584 / max: 10936.896) us
compute_phi_lambda_phiT_flat:    CPU:  4685.293 us   +/- 912.687 (min:  2661.400 / max:  6155.700) us     GPU-0:  5434.447 us   +/- 979.501 (min:  3005.440 / max:  7071.360) us
compute_phi_lambda_phiT_flat_all:    CPU:  6776.033 us   +/- 7288.636 (min:

In [154]:
# Generate random test data
phi_tilde = np.random.randn(100, 500, 4, 2)
# Lambda matrix
eps = np.random.randn(2, 500)

In [162]:
@njit(fastmath=True)
def compute_cov_matrices(epsilon):
    d, N = epsilon.shape

    # Step 1: Compute Lambda (covariance matrix)
    # Lambda = 1 / (N-1) * epsilon_centered @ epsilon_centered.T

    # First, compute mean of epsilon over N
    mean_epsilon = np.zeros(d)
    for i in range(d):
        for n in range(N):
            mean_epsilon[i] += epsilon[i, n]
        mean_epsilon[i] /= N

    # Compute centered epsilon
    epsilon_centered = np.zeros((d, N))
    for i in range(d):
        for n in range(N):
            epsilon_centered[i, n] = epsilon[i, n] - mean_epsilon[i]

    # Now compute epsilon_centered @ epsilon_centered.T
    Lambda = np.zeros((d, d))
    for i in range(d):
        for j in range(d):
            sum_ = 0.0
            for n in range(N):
                sum_ += epsilon_centered[i, n] * epsilon_centered[j, n]
            Lambda[i, j] = sum_ / (N - 1)


    # Step 2: Compute Lambda_n = 1/N * epsilon @ epsilon.T
    Lambda_n = np.zeros((d, d))
    for i in range(d):
        for j in range(d):
            sum_ = 0.0
            for n in range(N):
                sum_ += epsilon[i, n] * epsilon[j, n]
            Lambda_n[i, j] = sum_ / N

    # Step 3: Compute Lambda_inv
    Lambda_inv = np.linalg.inv(Lambda)

    # Step 4: Compute result = Lambda_inv @ Lambda_n @ Lambda_inv
    # First compute temp = Lambda_inv @ Lambda_n
    temp = np.zeros((d, d))
    for i in range(d):
        for j in range(d):
            for k in range(d):
                temp[i, j] += Lambda_inv[i, k] * Lambda_n[k, j]

    # Now compute result = temp @ Lambda_inv
    result = np.zeros((d, d))
    for i in range(d):
        for j in range(d):
            for k in range(d):
                result[i, j] += temp[i, k] * Lambda_inv[k, j]

    return Lambda_inv, result


@njit(cache=True, fastmath=True, parallel=True)
def compute_R(epsilon, phi_tilde):
    """
    Computes the R matrix using the covariance matrices and phi_tilde.
    R = sqrt(Delat(lambda^{-1})^{-1} @ Delta(lambda^{-1}lambda_n lambda^{-1}) @ Delta(lambda^{-1})^{-1})
    """
    Lambda_inv, Lambda_inv_n = compute_cov_matrices(epsilon)
    Delta_lambda = compute_phi_lambda_phiT_manual(phi_tilde, Lambda_inv)
    Delta_lambda_n = compute_phi_lambda_phiT_manual(phi_tilde, Lambda_inv_n)

    m, r, _ = Delta_lambda.shape

    result = np.zeros((m, r, r))
    
    for i in prange(m):
        Delta_inv = np.linalg.inv(Delta_lambda[i])
        temp = np.dot(Delta_inv, Delta_lambda_n[i])
        R = np.dot(temp, Delta_inv)
        result[i] = np.linalg.cholesky(R)

    return result

@njit(cache=True, fastmath=True, parallel=True)
def compute_R2(epsilon, phi_tilde):
    """
    Computes the R matrix using the covariance matrices and phi_tilde.
    R = sqrt(Delta(lambda^{-1})^{-1} @ Delta(lambda^{-1} lambda_n lambda^{-1}) @ Delta(lambda^{-1})^{-1})
    """
    Lambda_inv, Lambda_inv_n = compute_cov_matrices(epsilon)
    Delta_lambda = compute_phi_lambda_phiT_manual(phi_tilde, Lambda_inv)
    Delta_lambda_n = compute_phi_lambda_phiT_manual(phi_tilde, Lambda_inv_n)

    m, r, _ = Delta_lambda.shape

    result = np.zeros((m, r, r))
    
    for i in prange(m):
        Delta_inv = np.linalg.inv(Delta_lambda[i])

        # Compute R = Delta_inv @ Delta_lambda_n[i] @ Delta_inv manually
        R = np.zeros((r, r))
        for j in range(r):
            for k in range(r):
                sum_ = 0.0
                for l in range(r):
                    Delta_inv_jl = Delta_inv[j, l]
                    Delta_lambda_n_il = Delta_lambda_n[i, l]
                    for p in range(r):
                        sum_ += Delta_inv_jl *  Delta_lambda_n_il[p] * Delta_inv[p, k]
                R[j, k] = sum_

        # Now compute the Cholesky decomposition
        result[i] = np.linalg.cholesky(R)

    return result


def compute_R_np(epsilon, phi_tilde):
    Lambda_inv, Lambda_inv_n = compute_cov_matrices(epsilon)
    Delta_lambda = compute_phi_lambda_phiT_manual(phi_tilde, Lambda_inv)
    Delta_lambda_n = compute_phi_lambda_phiT_manual(phi_tilde, Lambda_inv_n)

    Delta_lambda_inv = np.linalg.inv(Delta_lambda)
    
    # Delta_lambda_inv @ Delta_lambda_n @ Delta_lambda_inv
    result = np.matmul(np.matmul(Delta_lambda_inv, Delta_lambda_n), Delta_lambda_inv)
    return np.linalg.cholesky(result)



R = compute_R_np(eps, phi_tilde)
print(R.shape)
print(benchmark(compute_R_np, (eps, phi_tilde), n_repeat=100))
functions = [
    compute_R,
    compute_R2
]
for func in functions:
    R = func(eps, phi_tilde)
    print(f"{func.__name__}: ", np.allclose(R, R))
    print(benchmark(func, (eps, phi_tilde), n_repeat=100))


(100, 4, 4)
compute_R_np        :    CPU: 19912.876 us   +/- 4974.924 (min: 14720.100 / max: 44286.300) us     GPU-0: 20449.287 us   +/- 5085.501 (min: 15223.328 / max: 44843.006) us
compute_R:  True
compute_R           :    CPU:  8838.961 us   +/- 1596.744 (min:  5874.200 / max: 12011.600) us     GPU-0:  9606.922 us   +/- 1583.851 (min:  6603.808 / max: 12771.232) us
compute_R2:  True
compute_R2          :    CPU:  9043.895 us   +/- 2384.871 (min:  5683.100 / max: 19738.700) us     GPU-0:  9757.777 us   +/- 2396.750 (min:  6224.000 / max: 19707.968) us


In [203]:
# Generate random test data
phi_tilde = np.random.randn(100, 500, 4, 2)
# Lambda matrix
eps = np.random.randn(2, 500)
alpha = np.sign(np.random.randn(100, 2, 500))
alpha[0, :] = 1
N_hat_pertubed = np.multiply(alpha, eps)

In [200]:
@njit(cache=True, fastmath=True)
def compute_cov_matrices_2(epsilon: np.ndarray):
    d, N = epsilon.shape

    # Step 1: Compute Lambda (covariance matrix)
    # Lambda = 1 / (N-1) * epsilon_centered @ epsilon_centered.T

    # First, compute mean of epsilon over N
    mean_epsilon = np.zeros(d)
    for i in range(d):
        for n in range(N):
            mean_epsilon[i] += epsilon[i, n]
        mean_epsilon[i] /= N

    # Compute centered epsilon
    epsilon_centered = np.zeros((d, N))
    for i in range(d):
        for n in range(N):
            epsilon_centered[i, n] = epsilon[i, n] - mean_epsilon[i]

    # Now compute epsilon_centered @ epsilon_centered.T
    Lambda = np.zeros((d, d))
    for i in range(d):
        for j in range(d):
            sum_ = 0.0
            for n in range(N):
                sum_ += epsilon_centered[i, n] * epsilon_centered[j, n]
            Lambda[i, j] = sum_ / (N - 1)


    # Step 2: Compute Lambda_n = 1/N * epsilon @ epsilon.T
    Lambda_n = np.zeros((d, d))
    for i in range(d):
        for j in range(d):
            sum_ = 0.0
            for n in range(N):
                sum_ += epsilon[i, n] * epsilon[j, n]
            Lambda_n[i, j] = sum_ / N

    # Step 3: Compute Lambda_inv
    Lambda_inv = np.linalg.inv(Lambda)

    return Lambda_inv, Lambda_n


@njit(cache=True, fastmath=True, parallel=True)
def compute_phi_lambda_phiT_and_phi_lambda_Y(phi_tilde, Lambda_inv, Lambda_n, N_hat_perturbed):
    """
    Computes:
      - result1 = phi @ Lambda_inv @ phi^T (averaged over T)
      - result2 = phi @ Lambda_inv @ Lambda_n @ Lambda_inv @ phi^T (averaged over T)
      - result3 = phi @ Lambda_inv @ N_hat_perturbed (summed over T)
    """
    m, T, r, c = phi_tilde.shape
    result1 = np.zeros((m, r, r))
    result2 = np.zeros((m, r, r))
    result3 = np.zeros((m, r, 1))  # result for phi_Lambda @ N_hat_perturbed

    for k in prange(m):
        for t in range(T):
            phi = phi_tilde[k, t]  # (r, c)

            # Precompute phi_Lambda_inv
            phi_Lambda_inv = np.zeros((r, c))
            for i in range(r):
                for l in range(c):
                    for q in range(c):
                        phi_Lambda_inv[i, l] += phi[i, q] * Lambda_inv[q, l]

            # Compute result1 and result2
            for i in range(r):
                for j in range(r):
                    sum1 = 0.0
                    sum2 = 0.0
                    for l in range(c):
                        sum1 += phi_Lambda_inv[i, l] * phi[j, l]
                        for p in range(c):
                            sum2 += phi_Lambda_inv[i, l] * Lambda_n[l, p] * phi_Lambda_inv[j, p]
                    result1[k, i, j] += sum1
                    result2[k, i, j] += sum2

            # Compute result3 = phi_Lambda_inv @ N_hat_perturbed
            for i in range(r):
                acc = 0.0
                for l in range(c):
                    acc += phi_Lambda_inv[i, l] * N_hat_perturbed[k, l, t]
                result3[k, i, 0] += acc

    result1 /= T
    result2 /= T
    result3 /= T

    return result1, result2, result3

@njit(cache=True, fastmath=True, parallel=True)
def compute_phi_lambda_Y(phi_tilde, Lambda_inv, Y):
    m, T, r, c = phi_tilde.shape  # (m, T, r, c)
    result = np.zeros((m, r, 1))  # (m, r, 1)

    for i in prange(m):       # systems
        for j in range(T):   # time
            for k in range(r):     # output dim (r)
                acc = 0.0
                for l in range(c): # inner dim (c)
                    # Compute phi @ Lambda_inv @ Y directly
                    for p in range(c):  # Lambda_inv interaction with Y
                        acc += phi_tilde[i, j, k, l] * Lambda_inv[l, p] * Y[i, p, j]
                result[i, k, 0] += acc
    result /= T  # Average over T dimension
    return result

@njit(cache=True, fastmath=True, parallel=True)
def compute_phi_lambda_phiT(phi_tilde, Lambda):
    m, T, r, c = phi_tilde.shape  # (m, T, r, c)
    result = np.zeros((m, r, r))
    
    for k in prange(m):
        for t in range(T):
            phi = phi_tilde[k, t]  # (r, c)

            for i in range(r):
                for j in range(r):
                    sum_ = 0.0
                    for l in range(c):
                        for p in range(c):
                            sum_ += phi[i, l] * Lambda[l, p] * phi[j, p]
                    result[k, i, j] += sum_

    result /= T
    return result

@njit(cache=True, fastmath=True)
def compute_cov_matrices(epsilon: np.ndarray):
    d, N = epsilon.shape

    # Step 1: Compute Lambda (covariance matrix)
    # Lambda = 1 / (N-1) * epsilon_centered @ epsilon_centered.T

    # First, compute mean of epsilon over N
    mean_epsilon = np.zeros(d)
    for i in range(d):
        for n in range(N):
            mean_epsilon[i] += epsilon[i, n]
        mean_epsilon[i] /= N

    # Compute centered epsilon
    epsilon_centered = np.zeros((d, N))
    for i in range(d):
        for n in range(N):
            epsilon_centered[i, n] = epsilon[i, n] - mean_epsilon[i]

    # Now compute epsilon_centered @ epsilon_centered.T
    Lambda = np.zeros((d, d))
    for i in range(d):
        for j in range(d):
            sum_ = 0.0
            for n in range(N):
                sum_ += epsilon_centered[i, n] * epsilon_centered[j, n]
            Lambda[i, j] = sum_ / (N - 1)


    # Step 2: Compute Lambda_n = 1/N * epsilon @ epsilon.T
    Lambda_n = np.zeros((d, d))
    for i in range(d):
        for j in range(d):
            sum_ = 0.0
            for n in range(N):
                sum_ += epsilon[i, n] * epsilon[j, n]
            Lambda_n[i, j] = sum_ / N

    # Step 3: Compute Lambda_inv
    Lambda_inv = np.linalg.inv(Lambda)

    # Step 4: Compute result = Lambda_inv @ Lambda_n @ Lambda_inv
    # First compute temp = Lambda_inv @ Lambda_n
    temp = np.zeros((d, d))
    for i in range(d):
        for j in range(d):
            for k in range(d):
                temp[i, j] += Lambda_inv[i, k] * Lambda_n[k, j]

    # Now compute result = temp @ Lambda_inv
    result = np.zeros((d, d))
    for i in range(d):
        for j in range(d):
            for k in range(d):
                result[i, j] += temp[i, k] * Lambda_inv[k, j]

    return Lambda_inv, result

def test_func1(eps, phi_tilde, N_hat_perturbed):
    Lambda_inv, Lambda_inv_n = compute_cov_matrices(eps)
    Delta_lambda = compute_phi_lambda_phiT(phi_tilde, Lambda_inv)
    Delta_lambda_n = compute_phi_lambda_phiT(phi_tilde, Lambda_inv_n)
    Delta_lambda_Y = compute_phi_lambda_Y(phi_tilde, Lambda_inv, N_hat_perturbed)
    return Delta_lambda, Delta_lambda_n, Delta_lambda_Y

def test_func2(eps, phi_tilde, N_hat_perturbed):
    Lambda_inv, Lambda_n = compute_cov_matrices_2(eps)
    Delta_lambda, Delta_lambda_n, Delta_lambda_Y = compute_phi_lambda_phiT_and_phi_lambda_Y(phi_tilde, Lambda_inv, Lambda_n, N_hat_perturbed)
    return Delta_lambda, Delta_lambda_n, Delta_lambda_Y
import numpy as np

def compute_phi_lambda_Y_np(phi_tilde, Lambda_inv, Y):
    # phi_tilde: (m, T, r, c)
    # Lambda_inv: (c, c)
    # Y: (m, c, T)
    
    # Step 1: Compute phi @ Lambda_inv
    phi_lambda = np.matmul(phi_tilde, Lambda_inv)  # (m, T, r, c) @ (c, c) -> (m, T, r, c)
    
    # Step 2: Compute phi_lambda @ Y
    print(phi_lambda.shape, Y.transpose(0, 2, 1)[:,:,:,None].shape)
    result = np.matmul(phi_lambda, Y.transpose(0, 2, 1)[:,:,:,None])  # (m, T, r, c) @ (m, T, c) -> (m, T, r, 1)
    
    # Step 3: Average over time T
    result = result.mean(axis=1)  # Average over T dimension

    return result

Lambda_inv, Lambda_n = compute_cov_matrices_2(eps)
phi_y_np = compute_phi_lambda_Y_np(phi_tilde, Lambda_inv, N_hat_pertubed)
print(phi_y_np.shape)
d1, d2, y1 = test_func1(eps, phi_tilde, N_hat_pertubed)
d3, d4, y2 = test_func2(eps, phi_tilde, N_hat_pertubed)
print(np.allclose(d1, d3), np.allclose(d2, d4), np.allclose(y1, y2))
print(np.allclose(y1, phi_y_np), np.allclose(y2, phi_y_np))
print(benchmark(test_func1, (eps, phi_tilde, N_hat_pertubed), n_repeat=100))
print(benchmark(test_func2, (eps, phi_tilde, N_hat_pertubed), n_repeat=1000))

(100, 500, 4, 2) (100, 500, 2, 1)
(100, 4, 1)
True True True
True True
test_func1          :    CPU:  9051.784 us   +/- 1793.271 (min:  6071.300 / max: 13201.600) us     GPU-0:  9697.676 us   +/- 1843.813 (min:  6190.976 / max: 14249.440) us
test_func2          :    CPU:  6138.082 us   +/- 1485.951 (min:  3798.100 / max: 15255.300) us     GPU-0:  6831.649 us   +/- 1542.071 (min:  3860.128 / max: 16242.912) us


In [220]:
def compute_S_np(N_hat, N_hat_perturbed, phi_tilde):
    """
    Computes the R matrix using the covariance matrices and phi_tilde.
    R = sqrt(Delat(lambda^{-1})^{-1} @ Delta(lambda^{-1}lambda_n lambda^{-1}) @ Delta(lambda^{-1})^{-1})
    """
    Lambda_inv, Lambda_n = compute_cov_matrices_2(N_hat)
    Delta_lambda, Delta_lambda_n, Delta_lambda_Y = compute_phi_lambda_phiT_and_phi_lambda_Y(phi_tilde, Lambda_inv, Lambda_n, N_hat_perturbed)
    # Compute the R
    m, r, _ = Delta_lambda.shape
    S = np.zeros((m, ))
    for i in range(m):
        Delta_inv = np.linalg.inv(Delta_lambda[i])
        temp = np.dot(Delta_inv, Delta_lambda_n[i])
        R_ = np.dot(temp, Delta_inv)
        W = np.linalg.cholesky(R_)
        S_i = np.dot(W, Delta_lambda_Y[i])
        S[i] = np.linalg.norm(S_i, ord=2)
    return S

@njit(cache=True, fastmath=True, parallel=True)
def compute_S(N_hat, N_hat_perturbed, phi_tilde):
    """
    Compute ranking matrix S
    Step 1: Compute covariance matrices Lambda_inv and Lambda_n
    Step 2: Compute Delta_lambda, Delta_lambda_n, and Delta_lambda_Y using phi_tilde and covariance matrices
    Step 3: Compute weight matrix W using Cholesky decomposition of R
    Step 4: Compute S using the norm of the product of W and Delta_lambda_Y
    """
    Lambda_inv, Lambda_n = compute_cov_matrices_2(N_hat)
    Delta_lambda, Delta_lambda_n, Delta_lambda_Y = compute_phi_lambda_phiT_and_phi_lambda_Y(phi_tilde, Lambda_inv, Lambda_n, N_hat_perturbed)
    # Compute the R
    m, r, _ = Delta_lambda.shape
    S = np.zeros((m, ))
    for i in prange(m):
        Delta_inv = np.linalg.inv(Delta_lambda[i])
        temp = np.dot(Delta_inv, Delta_lambda_n[i])
        R_ = np.dot(temp, Delta_inv)
        W = np.linalg.cholesky(R_)
        S_i = np.dot(W, Delta_lambda_Y[i])
        S[i] = np.linalg.norm(S_i, ord=2)
    return S

@njit(cache=True, fastmath=True, parallel=True)
def compute_S_1(N_hat, N_hat_perturbed, phi_tilde):
    """
    Compute ranking matrix S using manual loops for matrix multiplications.
    Keep Cholesky decomposition with numpy.
    """
    Lambda_inv, Lambda_n = compute_cov_matrices_2(N_hat)
    Delta_lambda, Delta_lambda_n, Delta_lambda_Y = compute_phi_lambda_phiT_and_phi_lambda_Y(
        phi_tilde, Lambda_inv, Lambda_n, N_hat_perturbed
    )
    
    m, r, _ = Delta_lambda.shape
    S = np.zeros((m, ))

    for i in prange(m):
        # Step 1: Invert Delta_lambda[i] (still using numpy)
        Delta_inv = np.linalg.inv(Delta_lambda[i])

        # Step 2: temp = Delta_inv @ Delta_lambda_n[i]
        temp = np.zeros((r, r))
        for a in range(r):
            for b in range(r):
                for c in range(r):
                    temp[a, b] += Delta_inv[a, c] * Delta_lambda_n[i, c, b]

        # Step 3: R_ = temp @ Delta_inv
        R_ = np.zeros((r, r))
        for a in range(r):
            for b in range(r):
                for c in range(r):
                    R_[a, b] += temp[a, c] * Delta_inv[c, b]

        # Step 4: Cholesky decomposition using numpy
        W = np.linalg.cholesky(R_)

        # Step 5: S_i = W @ Delta_lambda_Y[i]
        S_i = np.zeros((r, 1))
        for a in range(r):
            for b in range(r):
                S_i[a, 0] += W[a, b] * Delta_lambda_Y[i, b, 0]

        # Step 6: Compute norm of S_i
        norm = 0.0
        for a in range(r):
            norm += S_i[a, 0] ** 2
        S[i] = np.sqrt(norm)

    return S

@njit(cache=True, fastmath=True, parallel=True)
def compute_S_2(N_hat, N_hat_perturbed, phi_tilde):
    """
    Compute ranking matrix S using manual loops for matrix multiplications.
    Keep Cholesky decomposition with numpy.
    """
    Lambda_inv, Lambda_n = compute_cov_matrices_2(N_hat)
    Delta_lambda, Delta_lambda_n, Delta_lambda_Y = compute_phi_lambda_phiT_and_phi_lambda_Y(
        phi_tilde, Lambda_inv, Lambda_n, N_hat_perturbed
    )
    
    m, r, _ = Delta_lambda.shape
    S = np.zeros((m, ))

    for i in prange(m):
        Delta_inv = np.linalg.inv(Delta_lambda[i])

        R_ = np.zeros((r, r))
        for a in range(r):
            for b in range(r):
                for c in range(r):
                    for d in range(r):
                        R_[a, b] += Delta_inv[a, c] * Delta_lambda_n[i, c, d] * Delta_inv[d, b]

        W = np.linalg.cholesky(R_)

        S_i = np.zeros((r, 1))
        for a in range(r):
            for b in range(r):
                S_i[a, 0] += W[a, b] * Delta_lambda_Y[i, b, 0]

        norm = 0.0
        for a in range(r):
            norm += S_i[a, 0] ** 2
        S[i] = np.sqrt(norm)


    return S

@njit(cache=True, fastmath=True)
def compute_R(Delta_inv, Delta_lambda_n, r):
    """
    Computes the R matrix using the covariance matrices and phi_tilde.
    R = sqrt(Delat(lambda^{-1})^{-1} @ Delta(lambda^{-1}lambda_n lambda^{-1}) @ Delta(lambda^{-1})^{-1})
    """
    R_ = np.zeros((r, r))
    for a in range(r):
        for b in range(r):
            for c in range(r):
                for d in range(r):
                    R_[a, b] += Delta_inv[a, c] * Delta_lambda_n[c, d] * Delta_inv[d, b]

    return R_

@njit(cache=True, fastmath=True, parallel=True)
def compute_S_3(N_hat, N_hat_perturbed, phi_tilde):
    """
    Compute ranking matrix S using manual loops for matrix multiplications.
    Keep Cholesky decomposition with numpy.
    """
    Lambda_inv, Lambda_n = compute_cov_matrices_2(N_hat)
    Delta_lambda, Delta_lambda_n, Delta_lambda_Y = compute_phi_lambda_phiT_and_phi_lambda_Y(
        phi_tilde, Lambda_inv, Lambda_n, N_hat_perturbed
    )
    
    m, r, _ = Delta_lambda.shape
    S = np.zeros((m, ))

    for i in prange(m):
        Delta_inv = np.linalg.inv(Delta_lambda[i])
        R_ = compute_R(Delta_inv, Delta_lambda_n[i], r)

        W = np.linalg.cholesky(R_)

        S_i = np.zeros((r, 1))
        for a in range(r):
            for b in range(r):
                S_i[a, 0] += W[a, b] * Delta_lambda_Y[i, b, 0]

        norm = 0.0
        for a in range(r):
            norm += S_i[a, 0] ** 2
        S[i] = np.sqrt(norm)


    return S

functions = [
    compute_S,
    compute_S_1,
    compute_S_2,
    compute_S_3,
]
S_np = compute_S_np(eps, N_hat_pertubed, phi_tilde)
print(benchmark(compute_S_np, (eps, N_hat_pertubed, phi_tilde), n_repeat=1000))
for func in functions:
    S = func(eps, N_hat_pertubed, phi_tilde)
    print(f"{func.__name__}: ", np.allclose(S, S_np))
    print(benchmark(func, (eps, N_hat_pertubed, phi_tilde), n_repeat=1000))



compute_S_np        :    CPU: 44457.008 us   +/- 8093.475 (min: 34074.700 / max: 171110.900) us     GPU-0: 44986.625 us   +/- 8115.925 (min: 35014.561 / max: 172020.538) us
compute_S:  True
compute_S           :    CPU:  6970.540 us   +/- 1485.662 (min:  4402.400 / max: 14675.600) us     GPU-0:  7661.185 us   +/- 1516.794 (min:  4146.528 / max: 14496.160) us
compute_S_1:  True
compute_S_1         :    CPU:  6866.046 us   +/- 1584.168 (min:  4303.400 / max: 21731.000) us     GPU-0:  7571.574 us   +/- 1625.275 (min:  4320.160 / max: 22451.168) us
compute_S_2:  True
compute_S_2         :    CPU:  6723.279 us   +/- 1434.315 (min:  4230.300 / max: 11547.800) us     GPU-0:  7442.503 us   +/- 1449.862 (min:  4351.776 / max: 12181.216) us
compute_S_3:  True
compute_S_3         :    CPU:  6795.107 us   +/- 1500.694 (min:  4190.500 / max: 15189.400) us     GPU-0:  7492.659 us   +/- 1537.404 (min:  4454.688 / max: 15988.352) us


In [221]:
for func in functions:
    S = func(eps, N_hat_pertubed, phi_tilde)
    print(benchmark(func, (eps, N_hat_pertubed, phi_tilde), n_repeat=10000))

compute_S           :    CPU:  7400.758 us   +/- 2151.702 (min:  4338.600 / max: 141018.900) us     GPU-0:  8072.979 us   +/- 2166.783 (min:  4219.232 / max: 141904.800) us
compute_S_1         :    CPU:  6848.320 us   +/- 2063.395 (min:  4121.000 / max: 142062.300) us     GPU-0:  7529.344 us   +/- 2072.630 (min:  3970.816 / max: 142766.235) us
compute_S_2         :    CPU:  7155.172 us   +/- 1841.936 (min:  4097.400 / max: 79509.800) us     GPU-0:  7826.180 us   +/- 1859.252 (min:  3787.232 / max: 80210.144) us
compute_S_3         :    CPU:  7524.164 us   +/- 2469.835 (min:  4320.900 / max: 157025.900) us     GPU-0:  8245.184 us   +/- 2508.270 (min:  3815.424 / max: 157813.339) us


### Archive

In [ ]:
@njit(cache=True, fastmath=True, parallel=True)
def compute_R(Lambda_inv, Lambda_inv_n , phi_tilde):
    """
    Computes the R matrix using the covariance matrices and phi_tilde.
    R = sqrt(Delat(lambda^{-1})^{-1} @ Delta(lambda^{-1}lambda_n lambda^{-1}) @ Delta(lambda^{-1})^{-1})
    """
    Delta_lambda = compute_phi_lambda_phiT(phi_tilde, Lambda_inv)
    Delta_lambda_n = compute_phi_lambda_phiT(phi_tilde, Lambda_inv_n)

    m, r, _ = Delta_lambda.shape

    result = np.zeros((m, r, r))
    
    for i in prange(m):
        Delta_inv = np.linalg.inv(Delta_lambda[i])
        temp = np.dot(Delta_inv, Delta_lambda_n[i])
        R = np.dot(temp, Delta_inv)
        result[i] = np.linalg.cholesky(R)

    return result

@njit(cache=True, fastmath=True)
def compute_cov_matrices(epsilon: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
    d, N = epsilon.shape

    # Step 1: Compute Lambda (covariance matrix)
    # Lambda = 1 / (N-1) * epsilon_centered @ epsilon_centered.T

    # First, compute mean of epsilon over N
    mean_epsilon = np.zeros(d)
    for i in range(d):
        for n in range(N):
            mean_epsilon[i] += epsilon[i, n]
        mean_epsilon[i] /= N

    # Compute centered epsilon
    epsilon_centered = np.zeros((d, N))
    for i in range(d):
        for n in range(N):
            epsilon_centered[i, n] = epsilon[i, n] - mean_epsilon[i]

    # Now compute epsilon_centered @ epsilon_centered.T
    Lambda = np.zeros((d, d))
    for i in range(d):
        for j in range(d):
            sum_ = 0.0
            for n in range(N):
                sum_ += epsilon_centered[i, n] * epsilon_centered[j, n]
            Lambda[i, j] = sum_ / (N - 1)


    # Step 2: Compute Lambda_n = 1/N * epsilon @ epsilon.T
    Lambda_n = np.zeros((d, d))
    for i in range(d):
        for j in range(d):
            sum_ = 0.0
            for n in range(N):
                sum_ += epsilon[i, n] * epsilon[j, n]
            Lambda_n[i, j] = sum_ / N

    # Step 3: Compute Lambda_inv
    Lambda_inv = np.linalg.inv(Lambda)

    # Step 4: Compute result = Lambda_inv @ Lambda_n @ Lambda_inv
    # First compute temp = Lambda_inv @ Lambda_n
    temp = np.zeros((d, d))
    for i in range(d):
        for j in range(d):
            for k in range(d):
                temp[i, j] += Lambda_inv[i, k] * Lambda_n[k, j]

    # Now compute result = temp @ Lambda_inv
    result = np.zeros((d, d))
    for i in range(d):
        for j in range(d):
            for k in range(d):
                result[i, j] += temp[i, k] * Lambda_inv[k, j]

    return Lambda_inv, result

@njit(cache=True, fastmath=True, parallel=True)
def compute_phi_lambda_phiT(phi_tilde, Lambda):
    m, T, r, c = phi_tilde.shape  # (m, T, r, c)
    result = np.zeros((m, r, r))
    
    for k in prange(m):
        for t in range(T):
            phi = phi_tilde[k, t]  # (r, c)

            for i in range(r):
                for j in range(r):
                    sum_ = 0.0
                    for l in range(c):
                        for p in range(c):
                            sum_ += phi[i, l] * Lambda[l, p] * phi[j, p]
                    result[k, i, j] += sum_

    result /= T
    return result


@njit(cache=True, fastmath=True)
def compute_phi_phiT(phi_tilde):
    m, T, r, c = phi_tilde.shape  # (3, 5, 4, 2)
    result = np.zeros((m, r, r))
    for k in range(m):
        for t in range(T):
            phi = phi_tilde[k, t]  # shape (4, 2)
            # phi @ phi.T -> (4, 4)
            for i in range(r):
                for j in range(r):
                    for l in range(c):  # inner dimension
                        result[k, i, j] += phi[i, l] * phi[j, l]
    result/=T
    return result

@njit(cache=True, fastmath=True)
def compute_phi_Y(phi, Y):
    m, t, r, c = phi.shape  # (3, 5, 4, 2)
    result = np.zeros((m, r, 1))  # (3, 4, 1)

    for i in range(m):       # systems
        for j in range(t):   # time
            for k in range(r):     # output dim (4)
                acc = 0.0
                for l in range(c): # inner dim (2)
                    acc += phi[i, j, k, l] * Y[i, l, j]  # note: Y is (m, 2, t)
                result[i, k, 0] += acc
    return result

### armax test

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal

class ARMAX:
    def __init__(self, A, B, C):
        self.A = np.array(A)
        self.B = np.array(B)
        self.C = np.array(C)
    
    def simulate_open_loop(self, n_samples, U=None, noise_std=0.1):
        Y = np.zeros(n_samples)
        N = np.random.normal(0, noise_std, n_samples)
        
        if U is None:
            U = np.zeros(n_samples)
        
        max_order = max(len(self.A), len(self.B), len(self.C))
        
        for t in range(max_order, n_samples):
            Y[t] = (- np.dot(self.A[1:], Y[t-1:t-len(self.A):-1]) 
                    + np.dot(self.B, U[t-1:t-len(self.B)-1:-1])
                    + np.dot(self.C, N[t:t-len(self.C):-1]))
        
        return Y, U, N

    def plot_results(self, Y, U, N, R=None):
        fig, axs = plt.subplots(4 if R is not None else 3, 1, figsize=(10, 10), sharex=True)
        
        axs[0].plot(Y)
        axs[0].set_ylabel('Output (Y)')
        axs[0].set_title('ARMAX Open-Loop Simulation Results')
        
        axs[1].plot(U)
        axs[1].set_ylabel('Input (U)')
        
        axs[2].plot(N)
        axs[2].set_ylabel('Noise (N)')
        
        if R is not None:
            axs[3].plot(R)
            axs[3].set_ylabel('Reference (R)')
            axs[3].set_xlabel('Time')
        else:
            axs[2].set_xlabel('Time')
        
        plt.tight_layout()
        plt.show()

# Example usage
A = [1, -0.33, 0.1]   # A(z^-1) = 1 - 0.33z^-1 + 0.1z^-2
B = [0.22, 0.1]       # B(z^-1) = 0.22z^-1 + 0.1z^-2
C = [1]               # 

armax_model = ARMAX(A, B, C)

n_samples = 100
U = 0.5 * signal.square(np.linspace(0, 10*np.pi, n_samples))  # External input signal

Y, U, noise = armax_model.simulate_open_loop(n_samples, U, noise_std=0.07)
armax_model.plot_results(Y, U, noise)  # No R needed for open-loop

Y_dot = np.gradient(Y)

U = U.reshape(1, -1)
Y = np.vstack([Y, Y_dot])
print(Y.shape, U.shape)


In [ ]:
params = np.array([-0.33, 0.1, 0.22, 0.1])
A_obs, B_obs, C_obs, D_obs, A,B = _construct_ss_from_params(params,C_obs)
G = d_tfs.ss_to_tf(A_obs, B_obs, C_obs, D_obs, check_assumption=False)
C = np.empty((n_output, 1))
H = np.zeros((n_output, n_output), dtype=object)
for i in range(n_output):
    C[i]=np.array([1.0])
    H[i,i]=d_tfs((np.array([1.0]),A))
print("A_obs: ", A_obs)
print("B_obs:",  B_obs)
print("C_obs: ", C_obs)
print("A",A)
print("B", B)
print(G)
print(H)
print(C)
open_loop_sps_mimo(G, H, A, B, C, Y, U)

## 2 input 2 output

In [ ]:
A = np.array([1, 0.3, -0.2])    # a1, a2
B = np.array([[0, 0.5, 0.1], [0, 0.8, 0.1]])     # b1, b2
C = np.array([[1, 0.5],[1, 0.8]])          # c1
